# SPEAR Manuscript Figures

Single-cell Prediction of gene Expression from ATAC-seq Regression.


## Prereqs

- Place run outputs under `output/results/spear_results/` (one subfolder per run with `models/` and metrics CSVs).

- Ensure logs (if used) are under `output/logs/` with `spear_*` naming.

- Keep `analysis/model_name_lookup.tsv` present (tracked in repo).

- Use the `spear_env` kernel with required deps installed (see README).

### How to run

1. Open this notebook from the repo root.

2. Adjust run include globs or paths if you want to target specific runs; otherwise leave defaults.

3. Run all cells top-to-bottom after outputs are in place to regenerate figures/CSVs.


## 1. Environment Setup


In [ ]:
from __future__ import annotations

from dataclasses import dataclass, replace
from datetime import datetime
from fnmatch import fnmatch
from pathlib import Path
import sys
project_root = Path.cwd().resolve()
while project_root.name in {'analysis', 'scripts'}:
    project_root = project_root.parent
src_root = project_root / 'src'
for candidate in (src_root, project_root):
    if str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))

from typing import Iterable, Optional, Sequence

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display, Image

# Configure plotting defaults for consistent styling
sns.set_theme(style="whitegrid", context="paper")
sns.set_context("paper", font_scale=1.1)
plt.rcParams.update({"figure.dpi": 160, "savefig.dpi": 320})
pd.options.display.max_columns = 120
pd.options.display.width = 180


In [ ]:
# Quick toggles (edit these only)
# - TARGET_DATASET: "embryonic", "endothelial", or "both" (default keeps both)
# - USE_1000PLUS100: include 1000-gene runs with 100-gene fallback
# - PREFER_1000_FALLBACK_100: keep most recent 1000-gene per model, fallback to 100-gene only if missing
GENE_COUNT = 1000  # Change to 100 for 100-gene analysis
TARGET_DATASET = "both"  # "embryonic", "endothelial", or "both"
USE_1000PLUS100 = True
PREFER_1000_FALLBACK_100 = True

# Build run filters from the toggles above.
if USE_1000PLUS100:
    gene_tokens = ["1000genes", "100genes"]
    gene_label = "1000plus100"
    gene_desc = "1000 genes (+ 100 genes backup)"
else:
    gene_tokens = [f"{GENE_COUNT}genes"]
    gene_label = f"{GENE_COUNT}genes"
    gene_desc = f"{GENE_COUNT} genes"

if TARGET_DATASET in ("embryonic", "endothelial"):
    RUN_GLOB_SELECTION = [f"*{TARGET_DATASET}*{token}*" for token in gene_tokens]
    RUN_SUBSET_LABEL = f"{TARGET_DATASET}_{gene_label}"
    RUN_SUBSET_DESCRIPTION = f"{TARGET_DATASET.title()} | {gene_desc}"
else:
    RUN_GLOB_SELECTION = [f"*{token}*" for token in gene_tokens]
    RUN_SUBSET_LABEL = f"all_{gene_label}"
    RUN_SUBSET_DESCRIPTION = f"All datasets | {gene_desc}"

REPORTS_SUFFIX = RUN_SUBSET_LABEL or f"{GENE_COUNT}genes"


## 2. Analysis Configuration


In [ ]:
# Centralised configuration for the notebook run
@dataclass
class AnalysisConfig:
    project_root: Path
    results_root: Path
    lookup_path: Path
    fig_dir: Path
    reports_dir: Path
    run_include_globs: tuple[str, ...] = ("*",)
    run_exclude: tuple[str, ...] = tuple()
    primary_split: str = "test"
    val_split: str = "val"
    train_split: str = "train"
    top_gene_count: int = 15
    top_model_count: int = 3
    random_seed: int = 7

    def __post_init__(self) -> None:
        # Create required directories if they are missing.
        self.fig_dir.mkdir(parents=True, exist_ok=True)
        self.reports_dir.mkdir(parents=True, exist_ok=True)
        self.lookup_path.parent.mkdir(parents=True, exist_ok=True)


project_root = Path.cwd().resolve()
while project_root.name in {"analysis", "scripts"}:
    project_root = project_root.parent

config = AnalysisConfig(
    project_root=project_root,
    results_root=project_root / "output" / "results" / "spear_results",
    lookup_path=project_root / "analysis" / "model_name_lookup.tsv",
    fig_dir=project_root / "analysis" / "figs" / "manuscript_figures",
    reports_dir=project_root / "analysis" / "reports" / f"manuscript_{REPORTS_SUFFIX}",
    run_include_globs=tuple(RUN_GLOB_SELECTION),
    run_exclude=tuple(),
    random_seed=7,
    top_gene_count=15,
    top_model_count=3,
)

if not config.results_root.exists():
    raise FileNotFoundError(f"Results directory missing: {config.results_root}")
if not config.lookup_path.exists():
    # Seed the lookup table if it is missing so later steps can append to it.
    pd.DataFrame({
        "model_id": [],
        "model_display_name": [],
        "model_short_name": [],
    }).to_csv(
        config.lookup_path, sep="	", index=False
    )

np.random.seed(config.random_seed)
rng = np.random.default_rng(config.random_seed)
FIGURES: dict[str, plt.Figure] = {}
TABLES: dict[str, pd.DataFrame] = {}
EXTRA_EXPORT_PATHS: list[tuple[Path, str]] = []
analysis_metadata: dict[str, object] = {
    "generated_at": datetime.now().isoformat(timespec="seconds"),
    "project_root": config.project_root,
    "results_root": config.results_root,
}



## 3. Run Discovery Utilities


In [ ]:
# Canonical representation of a single trained model output folder
@dataclass(frozen=True)
class RunRecord:
    run_name: str
    model_id: str
    run_path: Path
    model_path: Path
    metrics_path: Optional[Path]
    predictions_path: Optional[Path]
    training_history_path: Optional[Path]
    model_display: Optional[str] = None


LOOKUP_SPECIAL_CASES = {
    "cnn": "Convolutional Neural Network",
    "rnn": "Recurrent Neural Network",
    "lstm": "Long Short-Term Memory",
    "mlp": "Multilayer Perceptron",
    "svr": "Support Vector Regressor",
    "ols": "Ordinary Least Squares",
    "xgboost": "XGBoost",
    "catboost": "CatBoost",
    "hist_gradient_boosting": "Histogram Gradient Boosting",
    "extra_trees": "Extra Trees",
    "random_forest": "Random Forest",
    "elastic_net": "Elastic Net",
}

SHORT_NAME_FALLBACKS = {
    "Multilayer Perceptron": "MLP",
    "Graph Neural Network": "GNN",
    "Convolutional Neural Network": "CNN",
    "Long Short-Term Memory Network": "LSTM",
    "Recurrent Neural Network": "RNN",
    "Transformer Encoder": "Transformer",
    "Ordinary Least Squares": "OLS",
    "Extra Trees": "Extra Trees",
    "Random Forest": "Random Forest",
    "Ridge Regression": "Ridge",
}

MODEL_ID_TO_DISPLAY: dict[str, str] = {}
MODEL_ID_TO_SHORT: dict[str, str] = {}
MODEL_DISPLAY_TO_SHORT: dict[str, str] = {}


def _default_short_name(display_name: str) -> str:
    # Generate a lightweight abbreviation when none is provided.
    if not isinstance(display_name, str) or not display_name.strip():
        return ""
    tokens = [token for token in display_name.replace("(", " ").replace(")", " ").split() if token]
    if not tokens:
        return display_name
    acronym = "".join(token[0] for token in tokens if token and token[0].isalnum()).upper()
    if 1 < len(acronym) <= 5:
        return acronym
    return display_name


def _matches_any(value: str, patterns: Iterable[str]) -> bool:
    """Check if value matches any of the fnmatch patterns."""
    return any(fnmatch(value, pattern) for pattern in patterns)


def _first_existing(base_dir: Path, candidates: Sequence[str]) -> Optional[Path]:
    """Return the first candidate path that exists, or None."""
    for candidate in candidates:
        path = base_dir / candidate
        if path.exists():
            return path
    return None

In [ ]:
def discover_model_runs(
    results_root: Path,
    include_globs: Iterable[str],
    exclude_patterns: Iterable[str],
) -> list[RunRecord]:
    results_root = Path(results_root)
    if not results_root.exists():
        raise FileNotFoundError(f"Results root missing: {results_root}")

    records_by_key: dict[tuple[str, str], RunRecord] = {}
    include = tuple(include_globs) if include_globs else ("*",)
    exclude = tuple(exclude_patterns) if exclude_patterns else tuple()

    for models_dir in sorted(results_root.rglob("models")):
        if not models_dir.is_dir():
            continue
        run_dir = models_dir.parent
        run_name = run_dir.name
        if not _matches_any(run_name, include):
            continue
        if exclude and _matches_any(run_name, exclude):
            continue

        for model_dir in sorted(models_dir.iterdir()):
            if not model_dir.is_dir():
                continue

            model_id = model_dir.name
            metrics_path = _first_existing(
                model_dir,
                (
                    "metrics_per_gene.csv",
                    "metrics_by_gene.csv",
                    "metrics_cv.csv",
                ),
            )
            predictions_path = _first_existing(
                model_dir,
                (
                    "predictions_raw.csv",
                    "predictions.csv",
                ),
            )
            history_path = _first_existing(
                model_dir,
                (
                    "training_history.csv",
                    "training_history_loss.csv",
                ),
            )

            rec = RunRecord(
                run_name=run_name,
                model_id=model_id,
                run_path=run_dir,
                model_path=model_dir,
                metrics_path=metrics_path,
                predictions_path=predictions_path,
                training_history_path=history_path,
            )

            key = (run_name, model_id)
            existing = records_by_key.get(key)
            if existing is None:
                records_by_key[key] = rec
                continue

            existing_has_metrics = existing.metrics_path is not None and Path(existing.metrics_path).exists()
            current_has_metrics = metrics_path is not None and Path(metrics_path).exists()

            if existing_has_metrics and not current_has_metrics:
                # Keep the version that already has metrics
                continue
            if current_has_metrics and not existing_has_metrics:
                # Prefer the version with metrics when the existing one is missing
                records_by_key[key] = rec
                continue
            # If both have or both lack metrics, keep the first-seen (sorted iteration is stable)

    return list(records_by_key.values())

In [ ]:
def _read_lookup_table(path: Path) -> pd.DataFrame:
    path = Path(path)
    if not path.exists():
        return pd.DataFrame({
            "model_id": pd.Series(dtype="string"),
            "model_display_name": pd.Series(dtype="string"),
            "model_short_name": pd.Series(dtype="string"),
        })
    df = pd.read_csv(path, sep="\t")
    expected = {"model_id", "model_display_name"}
    missing_cols = expected.difference(df.columns)
    if missing_cols:
        raise ValueError(f"Lookup table missing required columns: {sorted(missing_cols)}")
    if "model_short_name" not in df.columns:
        df["model_short_name"] = df["model_display_name"].map(_default_short_name)
    else:
        df["model_short_name"] = df["model_short_name"].fillna("")
        missing_short = df["model_short_name"].str.strip() == ""
        df.loc[missing_short, "model_short_name"] = df.loc[missing_short, "model_display_name"].map(_default_short_name)
    return df


def _update_model_name_maps(df: pd.DataFrame) -> None:
    # Cache name lookups for downstream plotting helpers.
    global MODEL_ID_TO_DISPLAY, MODEL_ID_TO_SHORT, MODEL_DISPLAY_TO_SHORT
    if df.empty:
        MODEL_ID_TO_DISPLAY = {}
        MODEL_ID_TO_SHORT = {}
        MODEL_DISPLAY_TO_SHORT = {}
        return
    standardised = df.fillna("")
    MODEL_ID_TO_DISPLAY = {
        row.model_id: row.model_display_name or _guess_display_name(row.model_id)
        for row in standardised.itertuples(index=False)
    }
    MODEL_ID_TO_SHORT = {
        row.model_id: (row.model_short_name or MODEL_ID_TO_DISPLAY[row.model_id])
        for row in standardised.itertuples(index=False)
    }
    MODEL_DISPLAY_TO_SHORT = {
        MODEL_ID_TO_DISPLAY[row.model_id]: MODEL_ID_TO_SHORT[row.model_id]
        for row in standardised.itertuples(index=False)
    }


def _guess_display_name(model_id: str) -> str:
    if model_id in LOOKUP_SPECIAL_CASES:
        return LOOKUP_SPECIAL_CASES[model_id]
    parts = [part for part in model_id.replace("-", " ").replace("_", " ").split(" ") if part]
    if not parts:
        return model_id
    formatted = []
    for token in parts:
        if len(token) <= 3:
            formatted.append(token.upper())
        else:
            formatted.append(token.capitalize())
    return " ".join(formatted)


def ensure_model_lookup(path: Path, model_ids: Iterable[str]) -> pd.DataFrame:
    path = Path(path)
    df = _read_lookup_table(path)
    existing = set(df["model_id"]) if not df.empty else set()
    new_rows = []
    for model_id in sorted(set(model_ids).difference(existing)):
        display_name = _guess_display_name(model_id)
        short_name = SHORT_NAME_FALLBACKS.get(display_name, _default_short_name(display_name))
        new_rows.append(
            {
                "model_id": model_id,
                "model_display_name": display_name,
                "model_short_name": short_name,
            }
        )
    if new_rows:
        additions = pd.DataFrame(new_rows)
        df = pd.concat([df, additions], ignore_index=True) if not df.empty else additions
        df.sort_values("model_id", inplace=True)
        df.to_csv(path, sep="\t", index=False)
    return df


def attach_lookup(records: Sequence[RunRecord], model_lookup: pd.DataFrame) -> list[RunRecord]:
    if model_lookup.empty:
        return list(records)
    display_map = dict(zip(model_lookup["model_id"], model_lookup["model_display_name"]))
    resolved: list[RunRecord] = []
    for record in records:
        display = display_map.get(record.model_id, _guess_display_name(record.model_id))
        resolved.append(replace(record, model_display=display))
    return resolved

In [ ]:
def run_records_to_frame(records: Sequence[RunRecord]) -> pd.DataFrame:
    if not records:
        return pd.DataFrame(
            columns=[
                "run_name",
                "model_id",
                "model_display",
                "run_path",
                "model_path",
                "metrics_path",
                "predictions_path",
                "training_history_path",
            ]
        )
    data = [
        {
            "run_name": r.run_name,
            "model_id": r.model_id,
            "model_display": r.model_display or _guess_display_name(r.model_id),
            "run_path": r.run_path,
            "model_path": r.model_path,
            "metrics_path": r.metrics_path,
            "predictions_path": r.predictions_path,
            "training_history_path": r.training_history_path,
        }
        for r in records
    ]
    return pd.DataFrame(data)


def to_relative_path(path_like: Optional[Path], root: Path) -> Optional[str]:
    if path_like is None:
        return None
    path = Path(path_like)
    try:
        return str(path.resolve().relative_to(root))
    except Exception:
        return str(path.resolve())

In [ ]:
def load_metrics(records: Sequence[RunRecord]) -> tuple[pd.DataFrame, pd.DataFrame]:
    long_frames: list[pd.DataFrame] = []
    for record in records:
        metrics_path = record.metrics_path
        if metrics_path is None or not metrics_path.exists():
            continue
        try:
            df = pd.read_csv(metrics_path)
        except Exception as exc:
            print(f"Failed to load metrics from {metrics_path}: {exc}")
            continue
        required_cols = {"gene", "split", "pearson"}
        if not required_cols.issubset(df.columns):
            continue
        df = df.copy()
        df["run_name"] = record.run_name
        df["model_id"] = record.model_id
        df["model_display"] = record.model_display or _guess_display_name(record.model_id)
        long_frames.append(df)
    if not long_frames:
        return pd.DataFrame(), pd.DataFrame()
    metrics_long = pd.concat(long_frames, ignore_index=True)
    base_cols = [col for col in ["run_name", "model_id", "model_display", "gene", "split"] if col in metrics_long.columns]
    metric_cols = [col for col in metrics_long.columns if col not in base_cols]
    metrics_long = metrics_long[base_cols + metric_cols]

    wide = metrics_long.pivot_table(
        index=["run_name", "model_id", "model_display", "gene"],
        columns="split",
        values="pearson",
    )
    wide.columns = [f"{str(col).lower()}_pearson" for col in wide.columns]
    metrics_wide = wide.reset_index()

    return metrics_long, metrics_wide


def compute_model_summary(
    metrics_wide: pd.DataFrame,
    splits: Sequence[str] = ("train", "val", "test"),
) -> pd.DataFrame:
    if metrics_wide.empty:
        return pd.DataFrame(
            columns=[
                "model_display",
                "model_id",
                "run_name",
                *[f"{split}_pearson_mean" for split in splits],
                *[f"{split}_pearson_std" for split in splits],
            ]
        )
    summaries = []
    lower_splits = [split.lower() for split in splits]
    for (run_name, model_id, model_display), group in metrics_wide.groupby(["run_name", "model_id", "model_display"], dropna=False):
        row = {
            "run_name": run_name,
            "model_id": model_id,
            "model_display": model_display,
        }
        for split, lower in zip(splits, lower_splits):
            column = f"{lower}_pearson"
            if column in group:
                values = group[column].dropna()
                if not values.empty:
                    row[f"{split}_pearson_mean"] = values.mean()
                    row[f"{split}_pearson_std"] = values.std(ddof=1) if len(values) > 1 else float("nan")
        summaries.append(row)
    summary_df = pd.DataFrame(summaries)
    if "test_pearson_mean" in summary_df:
        summary_df.sort_values("test_pearson_mean", ascending=False, inplace=True)
    summary_df.set_index(["model_display", "model_id", "run_name"], inplace=True)
    return summary_df

In [ ]:
def maybe_store_table(store: dict[str, pd.DataFrame], key: str, table: pd.DataFrame) -> None:
    if table is None or table.empty:
        return
    store[key] = table


def register_figure(store: dict[str, object], key: str, fig: Optional[plt.Figure]) -> None:
    # Track generated matplotlib figures for later export.
    if fig is None:
        store.pop(key, None)
        return
    store[key] = fig


def compute_heatmap_limits(
    values: pd.DataFrame | np.ndarray,
    lower_percentile: float = 5.0,
    upper_percentile: float = 95.0,
    clip: tuple[float, float] = (0.0, 1.0),
    min_buffer: float = 0.01,
) -> tuple[float, float]:
    # Derive consistent vmin/vmax bounds so heatmaps emphasise the dense value range.
    data = np.asarray(values, dtype=float)
    data = data[np.isfinite(data)]
    if data.size == 0:
        return clip
    lower = np.percentile(data, lower_percentile)
    upper = np.percentile(data, upper_percentile)
    buffer = max(min_buffer, (upper - lower) * 0.05)
    vmin = max(clip[0], lower - buffer)
    vmax = min(clip[1], upper + buffer)
    if vmin > vmax:
        if clip[0] <= clip[1]:
            vmin, vmax = clip
        else:
            vmin, vmax = float(data.min()), float(data.max())
    if np.isclose(vmin, vmax):
        spread = max(min_buffer, abs(vmin) * 0.1 or min_buffer)
        vmin -= spread
        vmax += spread
        vmin = max(clip[0], vmin)
        vmax = min(clip[1], vmax)
        if vmin > vmax:
            vmin, vmax = float(data.min()), float(data.max())
    return vmin, vmax


def to_short_name(name: str | None) -> str:
    # Return a concise display name for figure titles and filenames.
    if not name:
        return ""
    if name in MODEL_DISPLAY_TO_SHORT:
        return MODEL_DISPLAY_TO_SHORT[name]
    return SHORT_NAME_FALLBACKS.get(name, name)

## 4. Discover and Inspect Run Metadata


In [ ]:
raw_run_records = discover_model_runs(
    config.results_root,
    config.run_include_globs,
    config.run_exclude,
)
model_lookup = ensure_model_lookup(
    config.lookup_path,
    [record.model_id for record in raw_run_records],
)
_update_model_name_maps(model_lookup)
run_records = attach_lookup(raw_run_records, model_lookup)

# Helper to identify dataset from a run name

def _dataset_from_run_name(name: str) -> Optional[str]:
    name = str(name).lower()
    if "embryonic" in name:
        return "Embryonic"
    if "endothelial" in name:
        return "Endothelial"
    return None

import re


def _gene_count_from_run_name(name: str) -> Optional[int]:
    match = re.search(r"(\d+)genes", str(name).lower())
    if not match:
        return None
    try:
        return int(match.group(1))
    except ValueError:
        return None


def _run_timestamp_key(name: str) -> tuple[int, int]:
    match = re.search(r"_(\d{8})_(\d{6})", str(name))
    if not match:
        return (0, 0)
    return (int(match.group(1)), int(match.group(2)))


def _has_metrics(rec: RunRecord) -> bool:
    path = rec.metrics_path
    if not path:
        return False
    path = path if isinstance(path, Path) else Path(path)
    return path.exists()

def _model_dir_has_files(rec: RunRecord) -> bool:
    path = rec.model_path
    if not path:
        return False
    path = path if isinstance(path, Path) else Path(path)
    try:
        return path.exists() and any(path.iterdir())
    except Exception:
        return False


def _is_nonempty_record(rec: RunRecord) -> bool:
    return _has_metrics(rec) or _model_dir_has_files(rec)



def _select_latest_runs(records: list[RunRecord]) -> list[RunRecord]:
    if not records:
        return []
    grouped: dict[tuple[str, int, str], list[RunRecord]] = {}
    passthrough: list[RunRecord] = []
    for rec in records:
        dataset = _dataset_from_run_name(rec.run_name)
        gene_count = _gene_count_from_run_name(rec.run_name)
        if not dataset or gene_count is None:
            passthrough.append(rec)
            continue
        key = (dataset, gene_count, rec.model_id)
        grouped.setdefault(key, []).append(rec)

    selected: list[RunRecord] = []
    for group in grouped.values():
        with_metrics = [rec for rec in group if _has_metrics(rec)]
        if not with_metrics:
            continue
        with_metrics.sort(key=lambda rec: _run_timestamp_key(rec.run_name), reverse=True)
        selected.append(with_metrics[0])

    return selected + passthrough


run_records = _select_latest_runs(list(run_records))

# Fallback: if analyzing 1000 genes and a specific (run, model) combo has missing metrics,
# try the 100-gene variant from archive. Only use fallback when needed.
FALLBACK_USED: set[tuple[str, str]] = set()
FALLBACK_DISPLAY_RENAME: dict[tuple[str, str], str] = {}
FALLBACK_COMBOS: set[tuple[str, str]] = set()  # (Dataset, model_id)
IGNORED_MISSING: set[tuple[str, str]] = set()
if GENE_COUNT == 1000:
    archive_root = config.results_root / "archive"
    if archive_root.exists():
        successful_1000: set[tuple[str, str]] = set()
        for record in raw_run_records:
            if "1000genes" not in record.run_name:
                continue
            metrics_path = record.metrics_path
            if metrics_path is None or not Path(metrics_path).exists():
                continue
            ds = _dataset_from_run_name(record.run_name)
            if ds:
                successful_1000.add((ds, record.model_id))

        for run_dir in archive_root.iterdir():
            if not run_dir.is_dir() or "1000genes" not in run_dir.name:
                continue
            ds = _dataset_from_run_name(run_dir.name)
            if not ds:
                continue
            models_dir = run_dir / "models"
            if not models_dir.exists():
                continue
            for model_dir in models_dir.iterdir():
                if not model_dir.is_dir():
                    continue
                metrics_path = _first_existing(model_dir, ("metrics_per_gene.csv", "metrics_by_gene.csv", "metrics_cv.csv"))
                if metrics_path and metrics_path.exists():
                    successful_1000.add((ds, model_dir.name))

        updated_records: list[RunRecord | None] = list(run_records)

        # Only replace missing 1000-gene records with 100-gene variants
        for idx, rec in enumerate(list(run_records)):
            metrics_path = rec.metrics_path
            metrics_missing = metrics_path is None or not Path(metrics_path).exists()
            if not metrics_missing:
                continue

            dataset_label = _dataset_from_run_name(rec.run_name)

            # Prefer another 1000-gene run in archive for the same dataset/model.
            alt_model_dir = None
            alt_metrics = None
            alt_run_dir = None
            candidates = [d for d in archive_root.iterdir() if d.is_dir() and "1000genes" in d.name]
            if dataset_label:
                candidates = [d for d in candidates if dataset_label.lower() in d.name.lower()]
            for cand in sorted(candidates, key=lambda d: _run_timestamp_key(d.name), reverse=True):
                model_dir = cand / "models" / rec.model_id
                if not model_dir.exists():
                    continue
                alt_metrics = _first_existing(model_dir, ("metrics_per_gene.csv", "metrics_by_gene.csv", "metrics_cv.csv"))
                if alt_metrics and alt_metrics.exists():
                    alt_run_dir = cand
                    alt_model_dir = model_dir
                    break

            if alt_run_dir is None and (dataset_label, rec.model_id) not in successful_1000:
                # Candidate alt run name by replacing gene count portion
                alt_run_name = rec.run_name.replace(f"{GENE_COUNT}genes", "100genes")
                alt_run_dir = archive_root / alt_run_name

                # If exact archive path not present, search for any archive run that contains
                # '100genes' and a matching model id under its models/ directory, filtered by dataset
                if not alt_run_dir.exists():
                    candidates = [d for d in archive_root.iterdir() if d.is_dir() and "100genes" in d.name]
                    if dataset_label:
                        candidates = [d for d in candidates if dataset_label.lower() in d.name.lower()]
                    found = None
                    for cand in sorted(candidates, key=lambda d: _run_timestamp_key(d.name), reverse=True):
                        model_dir = cand / "models" / rec.model_id
                        if not model_dir.exists():
                            continue
                        alt_metrics = _first_existing(model_dir, ("metrics_per_gene.csv", "metrics_by_gene.csv", "metrics_cv.csv"))
                        if alt_metrics and alt_metrics.exists():
                            found = (cand, model_dir, alt_metrics)
                            break
                    if found:
                        alt_run_dir, alt_model_dir, alt_metrics = found
                    else:
                        alt_model_dir = None
                        alt_metrics = None
                else:
                    alt_model_dir = alt_run_dir / "models" / rec.model_id
                    if alt_model_dir.exists():
                        alt_metrics = _first_existing(alt_model_dir, ("metrics_per_gene.csv", "metrics_by_gene.csv", "metrics_cv.csv"))

            if alt_metrics and alt_metrics.exists() and alt_model_dir and alt_model_dir.exists():
                alt_preds = _first_existing(
                    alt_model_dir,
                    (
                        "predictions_raw.csv",
                        "predictions.csv",
                    ),
                )
                alt_history = _first_existing(
                    alt_model_dir,
                    (
                        "training_history.csv",
                        "training_history_loss.csv",
                    ),
                )

                # Keep actual run_path/model_path pointing to archive files, but prepare
                # a user-friendly display name that normalises 'grn' -> 'spear' when present
                alt_run_dir_name = alt_run_dir.name
                display_run_name = alt_run_dir_name.replace("grn", "spear") if "grn" in alt_run_dir_name else alt_run_dir_name

                new_rec = RunRecord(
                    run_name=alt_run_dir_name,
                    model_id=rec.model_id,
                    run_path=alt_run_dir,
                    model_path=alt_model_dir,
                    metrics_path=alt_metrics,
                    predictions_path=alt_preds,
                    training_history_path=alt_history,
                    model_display=rec.model_display,
                )
                # Replace the entry and record that a fallback was used for this run+model
                updated_records[idx] = new_rec
                FALLBACK_USED.add((new_rec.run_name, new_rec.model_id))
                FALLBACK_DISPLAY_RENAME[(new_rec.run_name, new_rec.model_id)] = display_run_name
                dataset_label = dataset_label or _dataset_from_run_name(alt_run_dir_name)
                if dataset_label:
                    FALLBACK_COMBOS.add((dataset_label, rec.model_id))

        run_records = [r for r in updated_records if r is not None]

# Ensure model_display is attached for any replaced records.
run_records = attach_lookup(run_records, model_lookup)

# Add archive 100-gene runs when 1000-gene models are missing (per dataset)
if USE_1000PLUS100:
    archive_root = config.results_root / "archive"
    if archive_root.exists():
        dataset_models_1000 = {}
        for rec in run_records:
            if "1000genes" not in rec.run_name:
                continue
            ds = _dataset_from_run_name(rec.run_name) or ""
            dataset_models_1000.setdefault(ds, set()).add(rec.model_id)

        archive_candidates = {}
        for run_dir in archive_root.iterdir():
            if not run_dir.is_dir() or "100genes" not in run_dir.name:
                continue
            ds = _dataset_from_run_name(run_dir.name) or ""
            if not ds:
                continue
            models_dir = run_dir / "models"
            if not models_dir.exists():
                continue
            for model_dir in models_dir.iterdir():
                if not model_dir.is_dir():
                    continue
                metrics_path = _first_existing(model_dir, ("metrics_per_gene.csv", "metrics_by_gene.csv", "metrics_cv.csv"))
                if not metrics_path or not metrics_path.exists():
                    continue
                key = (ds, model_dir.name)
                archive_candidates.setdefault(key, []).append((run_dir, model_dir, metrics_path))

        def _timestamp_key(name: str) -> tuple[int, int]:
            import re
            match = re.search(r"_(\d{8})_(\d{6})", name)
            if not match:
                return (0, 0)
            return (int(match.group(1)), int(match.group(2)))

        added_records = []
        for (ds, model_id), candidates in archive_candidates.items():
            if model_id in dataset_models_1000.get(ds, set()):
                continue
            candidates.sort(key=lambda item: _timestamp_key(item[0].name), reverse=True)
            run_dir, model_dir, metrics_path = candidates[0]
            preds_path = _first_existing(model_dir, ("predictions_raw.csv", "predictions.csv"))
            history_path = _first_existing(model_dir, ("training_history.csv", "training_history_loss.csv"))
            new_rec = RunRecord(
                run_name=run_dir.name,
                model_id=model_id,
                run_path=run_dir,
                model_path=model_dir,
                metrics_path=metrics_path,
                predictions_path=preds_path,
                training_history_path=history_path,
                model_display=MODEL_ID_TO_DISPLAY.get(model_id, _guess_display_name(model_id)),
            )
            added_records.append(new_rec)
            FALLBACK_USED.add((new_rec.run_name, new_rec.model_id))
            if ds:
                FALLBACK_COMBOS.add((ds, model_id))
        if added_records:
            run_records = list(run_records) + added_records

# Prefer most recent 1000-gene runs, fallback to 100-gene when missing.

def _select_preferred_runs(records: list[RunRecord], prefer_1000: bool) -> list[RunRecord]:
    if not records:
        return []
    if not prefer_1000:
        return list(records)

    def _timestamp_key(name: str) -> tuple[int, int]:
        match = re.search(r"_(\d{8})_(\d{6})", name)
        if not match:
            return (0, 0)
        return (int(match.group(1)), int(match.group(2)))

    def _has_metrics(rec: RunRecord) -> bool:
        path = rec.metrics_path
        if not path:
            return False
        path = path if isinstance(path, Path) else Path(path)
        return path.exists()

    def _dataset_key(rec: RunRecord) -> str:
        return _dataset_from_run_name(rec.run_name) or ""

    selected: list[RunRecord] = []
    by_model_dataset: dict[tuple[str, str], list[RunRecord]] = {}
    for record in records:
        key = (record.model_id, _dataset_key(record))
        by_model_dataset.setdefault(key, []).append(record)

    for (model_id, dataset_label), group in by_model_dataset.items():
        group_1000 = [rec for rec in group if "1000genes" in rec.run_name and _has_metrics(rec)]
        group_100 = [rec for rec in group if "100genes" in rec.run_name and _has_metrics(rec)]
        group_1000_all = [rec for rec in group if "1000genes" in rec.run_name]
        group_100_all = [rec for rec in group if "100genes" in rec.run_name]
        candidates = group_1000 or group_100 or group_1000_all or group_100_all or group
        candidates.sort(key=lambda rec: _timestamp_key(rec.run_name), reverse=True)
        selected.append(candidates[0])

    return selected

run_records = _select_preferred_runs(list(run_records), PREFER_1000_FALLBACK_100)

run_df = run_records_to_frame(run_records)
run_df.sort_values(["run_name", "model_id"], inplace=True)
run_df_display = run_df.copy()
for column in ("run_path", "model_path", "metrics_path", "predictions_path", "training_history_path"):
    run_df_display[column] = run_df_display[column].map(lambda value: to_relative_path(value, config.project_root))

# Optionally rename display run names from 'grn' -> 'spear' when applicable for clarity.
if FALLBACK_DISPLAY_RENAME:
    mask = run_df_display.apply(lambda r: (r['run_name'], r['model_id']) in FALLBACK_DISPLAY_RENAME, axis=1)
    if mask.any():
        def _maybe_rename_row(r):
            key = (r['run_name'], r['model_id'])
            return FALLBACK_DISPLAY_RENAME.get(key, r['run_name'])
        run_df_display.loc[mask, 'run_name'] = run_df_display.loc[mask].apply(_maybe_rename_row, axis=1)

analysis_metadata.update({
    "results_root": to_relative_path(config.results_root, config.project_root),
    "fig_dir": to_relative_path(config.fig_dir, config.project_root),
    "reports_dir": to_relative_path(config.reports_dir, config.project_root),
    "run_count": run_df["run_name"].nunique(),
    "model_count": len(run_df),
    "model_lookup_path": to_relative_path(config.lookup_path, config.project_root),
})
analysis_metadata.setdefault("include_globs", config.run_include_globs)
analysis_metadata.setdefault("subset_descriptor", RUN_SUBSET_DESCRIPTION)
analysis_metadata.setdefault("subset_label", RUN_SUBSET_LABEL)

display(Markdown(f"**Scanning results root:** `{analysis_metadata['results_root']}`"))
include_filters = analysis_metadata.get("include_globs")
if include_filters:
    include_text = ", ".join(str(item) for item in include_filters)
    display(Markdown(f"**Include filters:** `{include_text}`"))
subset_descriptor = analysis_metadata.get("subset_descriptor")
if subset_descriptor:
    display(Markdown(f"**Subset criteria:** {subset_descriptor}"))

display(Markdown(
    f"**Figure output:** `{analysis_metadata['fig_dir']}` | **Reports:** `{analysis_metadata['reports_dir']}`"
))
display(run_df_display)
print(
    "Discovered",
    analysis_metadata["model_count"],
    "model folders across",
    analysis_metadata["run_count"],
    "runs.",
)
print(f"Fallback replacements: {len(FALLBACK_USED)} model+dataset combos using 100-gene from archive")
if IGNORED_MISSING:
    print(f"Ignored missing runs (no fallback): {sorted(IGNORED_MISSING)}")


# Export fallback usage list for transparency (covers archive + in-run 100-gene fallbacks).

# Build availability lookup from raw records to detect missing 1000-gene metrics.
model_dataset_key = []
for record in raw_run_records:
    ds = _dataset_from_run_name(record.run_name) or ""
    model_dataset_key.append((record.model_id, ds, record.run_name, record.metrics_path))

fallback_rows = []
for record in run_records:
    ds = _dataset_from_run_name(record.run_name) or ""
    if "100genes" not in record.run_name:
        continue
    # Check if any 1000-gene run exists with metrics for the same model/dataset.
    has_1000_metrics = False
    for model_id, dataset_label, run_name, metrics_path in model_dataset_key:
        if model_id != record.model_id or dataset_label != ds:
            continue
        if "1000genes" not in run_name:
            continue
        if metrics_path is None:
            continue
        try:
            if Path(metrics_path).exists():
                has_1000_metrics = True
                break
        except Exception:
            continue
    if not has_1000_metrics:
        fallback_rows.append({
            "dataset": ds,
            "model_id": record.model_id,
            "run_name": record.run_name,
            "source_run": record.run_name,
        })

# Include explicit archive replacements, if any.
for run_name, model_id in sorted(FALLBACK_USED):
    dataset_label = _dataset_from_run_name(run_name) or ""
    source = FALLBACK_DISPLAY_RENAME.get((run_name, model_id), run_name)
    fallback_rows.append({
        "dataset": dataset_label,
        "model_id": model_id,
        "run_name": run_name,
        "source_run": source,
    })

fallback_report = Path(config.reports_dir) / "fallback_models_used.csv"
if not fallback_rows:
    fallback_df = pd.DataFrame(columns=["dataset", "model_id", "run_name", "source_run"])
else:
    fallback_df = pd.DataFrame(fallback_rows).drop_duplicates()
fallback_df.to_csv(fallback_report, index=False)
analysis_metadata["fallback_report"] = to_relative_path(fallback_report, config.project_root)


## 5. Load Metrics and Compute Summaries


In [ ]:
metrics_long, metrics_wide = load_metrics(run_records)

# Table: mean test Pearson for all runs with metrics
all_records = attach_lookup(raw_run_records, model_lookup)
all_metrics_long, all_metrics_wide = load_metrics(all_records)
if all_metrics_wide.empty or "test_pearson" not in all_metrics_wide:
    print("No test-set metrics available for full run table.")
else:
    mean_df = (
        all_metrics_wide.groupby(["run_name", "model_id"])
        ["test_pearson"]
        .mean()
        .reset_index()
        .rename(columns={"test_pearson": "test_pearson_mean"})
    )
    path_df = run_records_to_frame(all_records)[["run_name", "model_id", "model_path"]]
    all_run_mean_test_pearson = mean_df.merge(path_df, on=["run_name", "model_id"], how="left")
    all_run_mean_test_pearson["dataset"] = all_run_mean_test_pearson["run_name"].map(_dataset_from_run_name)
    all_run_mean_test_pearson["model_path"] = all_run_mean_test_pearson["model_path"].map(
        lambda value: to_relative_path(value, config.project_root)
    )
    display(all_run_mean_test_pearson.sort_values(["run_name", "model_id"]))
    TABLES["all_run_mean_test_pearson"] = all_run_mean_test_pearson
if metrics_long.empty:
    raise RuntimeError("No metrics available for plotting.")


# Helper reused across sections

def _dataset_from_run_name(name: str) -> Optional[str]:
    name = str(name).lower()
    if "embryonic" in name:
        return "Embryonic"
    if "endothelial" in name:
        return "Endothelial"
    return None

split_filter = {config.primary_split, config.val_split, config.train_split}
available_splits = sorted(metrics_long["split"].unique())
missing_splits = split_filter.difference(available_splits)
if missing_splits:
    print("Warning: the following splits are missing from metrics files:", sorted(missing_splits))

metrics_long["dataset"] = metrics_long["run_name"].map(_dataset_from_run_name)
if not metrics_wide.empty and "dataset" not in metrics_wide:
    metrics_wide["dataset"] = metrics_wide["run_name"].map(_dataset_from_run_name)

test_metrics = metrics_long[metrics_long["split"] == config.primary_split].copy()
val_metrics = metrics_long[metrics_long["split"] == config.val_split].copy()
train_metrics = metrics_long[metrics_long["split"] == config.train_split].copy()
summary_df = compute_model_summary(
    metrics_wide, [config.primary_split, config.val_split, config.train_split]
)
if summary_df.empty:
    raise RuntimeError("Unable to compute summary statistics from metrics.")

summary_reset = summary_df.reset_index()
summary_reset["dataset"] = summary_reset["run_name"].map(_dataset_from_run_name)
analysis_metadata["best_model_id"] = summary_reset.iloc[0]["model_id"]
analysis_metadata["best_model_display"] = summary_reset.iloc[0]["model_display"]
analysis_metadata["best_run_name"] = summary_reset.iloc[0]["run_name"]

# Keep model_display untouched; fallback markers are added in plots instead of renaming.
fallback_pairs = set(globals().get("FALLBACK_COMBOS", set()))

if "test_pearson_mean" in summary_reset:
    model_display_order = summary_reset.sort_values(
        by="test_pearson_mean", ascending=False
    )[["model_display", "model_id"]].drop_duplicates("model_id")["model_display"].tolist()
else:
    model_display_order = summary_reset["model_display"].tolist()

display(summary_reset)

# Print fallback details for user reference
if fallback_pairs:
    print(f"\n=== FALLBACK MODELS ({len(fallback_pairs)} combos using 100-gene archive) ===")
    fallback_sources = list(globals().get("FALLBACK_USED", set()))
    for dataset_name, model_id in sorted(fallback_pairs):
        source = next(
            (run for run, mid in fallback_sources if mid == model_id and dataset_name.lower() in run.lower()),
            None,
        )
        source_text = source or "archive"
        print(f"  {model_id:20s} + {dataset_name:12s} (from: {source_text})")

analysis_state = {
    "run_df": run_df,
    "metrics_long": metrics_long,
    "metrics_wide": metrics_wide,
    "test_metrics": test_metrics,
    "val_metrics": val_metrics,
    "train_metrics": train_metrics,
    "summary_df": summary_df,
    "summary_reset": summary_reset,
    "model_display_order": model_display_order,
    "model_short_name_map": MODEL_ID_TO_SHORT.copy(),
}


## 6. Supporting Tables


In [ ]:
summary_df = analysis_state["summary_df"]
summary_reset = analysis_state["summary_reset"]
metrics_wide = analysis_state["metrics_wide"]
metrics_long = analysis_state["metrics_long"]

if "dataset" not in metrics_long:
    def _dataset_from_run_name(name: str) -> Optional[str]:
        name = str(name).lower()
        if "embryonic" in name:
            return "Embryonic"
        if "endothelial" in name:
            return "Endothelial"
        return None
    metrics_long["dataset"] = metrics_long["run_name"].map(_dataset_from_run_name)

val_pearson_per_gene = pd.DataFrame()
if f"{config.val_split}_pearson" in metrics_wide:
    val_pearson_per_gene = metrics_wide.pivot_table(
        index="gene",
        columns="model_display",
        values=f"{config.val_split}_pearson",
    )

test_pearson_per_gene = pd.DataFrame()
if f"{config.primary_split}_pearson" in metrics_wide:
    test_pearson_per_gene = metrics_wide.pivot_table(
        index="gene",
        columns="model_display",
        values=f"{config.primary_split}_pearson",
    )

# Order genes from highest to lowest test Pearson using Embryonic scores when available
_gene_order = pd.Index([])
emb_test = metrics_long[
    (metrics_long["split"] == config.primary_split)
    & (metrics_long["dataset"] == "Embryonic")
]
if not emb_test.empty and "pearson" in emb_test:
    _gene_order = (
        emb_test.groupby("gene")["pearson"].mean().sort_values(ascending=False).index
    )
if not test_pearson_per_gene.empty and len(_gene_order) > 0:
    aligned = test_pearson_per_gene.index.intersection(_gene_order)
    remainder = test_pearson_per_gene.index.difference(_gene_order)
    test_pearson_per_gene = test_pearson_per_gene.loc[list(aligned) + list(remainder)]
    if not val_pearson_per_gene.empty:
        val_pearson_per_gene = val_pearson_per_gene.reindex(test_pearson_per_gene.index)

maybe_store_table(
    TABLES,
    "metrics_per_gene_master",
    metrics_long.sort_values(["split", "run_name", "model_id", "gene"]),
)
maybe_store_table(TABLES, "summary_metrics_all_models", summary_reset)

# Persist key summaries to disk
reports_dir = Path(config.reports_dir)
reports_dir.mkdir(parents=True, exist_ok=True)
summary_csv = reports_dir / "summary_metrics_all_models.csv"
summary_reset.to_csv(summary_csv, index=False)

# Write a compact markdown summary including fallbacks
results_md = reports_dir / "RESULTS_SUMMARY.md"
fallback_pairs = set(globals().get("FALLBACK_COMBOS", set()))
fallback_sources = list(globals().get("FALLBACK_USED", set()))
if fallback_pairs:
    fb_lines = ["- {} | {} | source: {}".format(
        dataset,
        model,
        next((run for run, mid in fallback_sources if mid == model and dataset.lower() in run.lower()), "archive"),
    ) for dataset, model in sorted(fallback_pairs)]
else:
    fb_lines = ["- none"]

_top = summary_reset.head(10)
md_table = "| " + " | ".join(_top.columns) + " |\n"
md_table += "| " + " | ".join(["---"] * len(_top.columns)) + " |\n"
for _, row in _top.iterrows():
    md_table += "| " + " | ".join(str(row[col]) for col in _top.columns) + " |\n"

md_text = "\n".join(
    ["# Results Summary",
     f"- Generated at: {analysis_metadata.get('generated_at', '')}",
     f"- Results root: {analysis_metadata.get('results_root', config.results_root)}",
     f"- Reports dir: {reports_dir}",
     f"- Run count: {analysis_metadata.get('run_count', len(summary_reset))}",
     f"- Model rows: {len(summary_reset)}",
     "- Fallbacks (dataset | model | source):",
     *fb_lines,
     "",
     "## Top models (test pearson)",
     md_table
    ]
)
results_md.write_text(md_text)

analysis_state.update(
    {
        "val_pearson_per_gene": val_pearson_per_gene,
        "test_pearson_per_gene": test_pearson_per_gene,
        "split_mean_summary": summary_reset,
        "summary_csv_path": summary_csv,
        "results_md_path": results_md,
    }
)


## 7. Heatmap: Embryonic vs Endothelial Models


In [ ]:
summary_reset = analysis_state["summary_reset"].copy()

def _assign_dataset(run_name: str) -> Optional[str]:
    name = str(run_name).lower()
    if "embryonic" in name:
        return "Embryonic"
    if "endothelial" in name:
        return "Endothelial"
    return None

if "dataset" not in summary_reset:
    summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)
else:
    summary_reset["dataset"] = summary_reset["dataset"].fillna(summary_reset["run_name"].map(_assign_dataset))

subset = summary_reset[summary_reset["dataset"].isin(["Embryonic", "Endothelial"])].copy()
fig_heatmap = None
title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"
if subset.empty or "test_pearson_mean" not in subset:
    print("Dataset-specific test Pearson summaries unavailable; skipping heatmap.")
else:
    ranked = subset.sort_values("test_pearson_mean", ascending=False)
    best_per_model = ranked.drop_duplicates(["dataset", "model_id"])
    heatmap_df = best_per_model.pivot_table(
        index="model_display",
        columns="dataset",
        values="test_pearson_mean",
    )
    if heatmap_df.empty:
        print("No values available for dataset heatmap.")
    else:
        if "Embryonic" in heatmap_df:
            order = heatmap_df["Embryonic"].sort_values(ascending=False).index
        else:
            order = heatmap_df.mean(axis=1).sort_values(ascending=False).index
        heatmap_df = heatmap_df.loc[order]
        column_order = [col for col in ["Embryonic", "Endothelial"] if col in heatmap_df.columns]
        heatmap_df = heatmap_df[column_order]
        vmin, vmax = compute_heatmap_limits(heatmap_df.values)
        fig_height = max(4, 0.4 * len(heatmap_df))
        fig_width = max(7, 2.0 * len(column_order))
        fig_heatmap, ax = plt.subplots(figsize=(fig_width, fig_height))
        sns.heatmap(
            heatmap_df,
            cmap="viridis",
            vmin=vmin,
            vmax=vmax,
            annot=True,
            fmt=".3f",
            linewidths=0.4,
            linecolor="#f2f2f2",
            cbar_kws={"label": "Mean Test Pearson"},
            ax=ax,
        )
        ax.set_title(f"Mean Test Pearson by Model and Dataset | {title_suffix}")
        ax.set_xlabel("")
        ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
        ax.tick_params(axis="x", pad=8)
        ax.set_ylabel("Model")
        sns.despine(fig_heatmap, left=True, bottom=True)
        plt.tight_layout()
        fig_heatmap.subplots_adjust(bottom=0.22)

register_figure(FIGURES, "test_pearson_heatmap_by_dataset", fig_heatmap)
if fig_heatmap is not None:
    display(fig_heatmap)
    plt.close(fig_heatmap)


## 8. Violin Plot: Test Pearson by Dataset


In [ ]:
metrics_long = analysis_state["metrics_long"].copy()
model_display_order = analysis_state["model_display_order"]
summary_reset = analysis_state.get("summary_reset", pd.DataFrame()).copy()
metrics_long["dataset"] = metrics_long["run_name"].map(_assign_dataset)

title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"

subset = metrics_long[
    (metrics_long["split"] == config.primary_split)
    & (metrics_long["dataset"].isin(["Embryonic", "Endothelial"]))
].copy()

fig_violin_by_dataset = None
if subset.empty:
    print("Test metrics unavailable for embryonic/endothelial comparison.")
else:
    emb_order = []
    if not summary_reset.empty and "test_pearson_mean" in summary_reset:
        emb_order = (
            summary_reset[summary_reset["dataset"] == "Embryonic"]
            .sort_values("test_pearson_mean", ascending=False)["model_display"].tolist()
        )
    ordered_models = [m for m in emb_order if m in subset["model_display"].unique()]
    if not ordered_models:
        ordered_models = [m for m in model_display_order if m in subset["model_display"].unique()]
    if not ordered_models:
        ordered_models = subset["model_display"].sort_values().unique().tolist()
    dataset_order = ["Embryonic", "Endothelial"]
    palette = {"Embryonic": "#4C78A8", "Endothelial": "#F58518"}
    fig_width = max(10, 0.75 * max(6, len(ordered_models)))
    fig_violin_by_dataset, ax = plt.subplots(figsize=(fig_width, 6.5))
    sns.violinplot(
        data=subset,
        x="model_display",
        y="pearson",
        hue="dataset",
        order=ordered_models,
        hue_order=dataset_order,
        palette=palette,
        density_norm="width",
        inner="quartile",
        linewidth=1.2,
        dodge=True,
        ax=ax,
    )
    # Emphasize quartile/median lines.
    for line in ax.lines:
        line.set_linewidth(2.0)
        line.set_color("#111111")

    jitter_sample = subset.sample(
        min(len(subset), 4000),
        random_state=config.random_seed,
    ) if len(subset) > 4000 else subset
    sns.stripplot(
        data=jitter_sample,
        x="model_display",
        y="pearson",
        hue="dataset",
        order=ordered_models,
        hue_order=dataset_order,
        palette=palette,
        dodge=True,
        jitter=0.12,
        size=2.4,
        alpha=0.35,
        edgecolor="#2b2b2b",
        linewidth=0.3,
        marker="o",
        ax=ax,
        legend=False,
    )

    ax.set_xticklabels(ordered_models, rotation=35, ha="right")
    ax.set_title(f"Test Pearson Distribution by Model and Dataset | {title_suffix}")
    ax.set_xlabel("Model")
    ax.set_ylabel("Per-gene Test Pearson")
    ax.legend(title="Dataset", bbox_to_anchor=(1.02, 1), loc="upper left")
    sns.despine(fig_violin_by_dataset, left=True, bottom=True)
    fig_violin_by_dataset.tight_layout()

register_figure(FIGURES, "test_violin_by_dataset", fig_violin_by_dataset)
if fig_violin_by_dataset is not None:
    display(fig_violin_by_dataset)
    plt.close(fig_violin_by_dataset)


## 9. Dataset-Level Summary Panels

Dataset-scoped metrics panels and generalization gap summaries.


In [ ]:
# Dataset-specific manuscript plots
summary_reset = analysis_state["summary_reset"].copy()
metrics_long = analysis_state["metrics_long"].copy()
metrics_wide = analysis_state["metrics_wide"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)
metrics_long["dataset"] = metrics_long["run_name"].map(_assign_dataset)
if not metrics_wide.empty and "dataset" not in metrics_wide:
    metrics_wide["dataset"] = metrics_wide["run_name"].map(_assign_dataset)

title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"
metrics_to_plot = {
    "pearson": "Pearson",
    "spearman": "Spearman",
    "r2": "R2",
    "rmse": "RMSE",
}

for dataset in ["Embryonic", "Endothelial"]:
    ds_summary = summary_reset[summary_reset["dataset"] == dataset].copy()
    ds_test = metrics_long[
        (metrics_long["dataset"] == dataset)
        & (metrics_long["split"] == config.primary_split)
    ].copy()
    ds_splits = metrics_long[
        (metrics_long["dataset"] == dataset)
        & (metrics_long["split"].isin([config.train_split, config.val_split, config.primary_split]))
    ].copy()

    if ds_summary.empty or ds_test.empty:
        print(f"Skipping {dataset} dataset-specific plots (no data).")
        continue

    model_order = (
        ds_summary.sort_values("test_pearson_mean", ascending=False)
        [["model_display", "model_id"]]
        .drop_duplicates("model_id")["model_display"]
        .tolist()
    )

    # Split comparison overview (per dataset)
    split_labels = {
        config.train_split: "Train",
        config.val_split: "Val",
        config.primary_split: "Test",
    }
    ds_splits = ds_splits[["model_display", "split", "pearson"]].dropna()
    ds_splits["split_label"] = ds_splits["split"].map(split_labels).fillna(ds_splits["split"].str.title())
    split_order = [split_labels[split] for split in [config.train_split, config.val_split, config.primary_split]]
    fig_width = max(12, 0.75 * max(6, len(model_order)))
    fig_split, ax = plt.subplots(figsize=(fig_width, 6.5))
    sns.boxplot(
        data=ds_splits,
        x="model_display",
        y="pearson",
        hue="split_label",
        order=model_order,
        hue_order=split_order,
        palette="Set2",
        width=0.65,
        fliersize=0,
        ax=ax,
    )
    jitter_sample = ds_splits.sample(min(len(ds_splits), 4000), random_state=config.random_seed) if len(ds_splits) > 4000 else ds_splits
    sns.stripplot(
        data=jitter_sample,
        x="model_display",
        y="pearson",
        hue="split_label",
        order=model_order,
        hue_order=split_order,
        palette="Set2",
        dodge=True,
        jitter=0.12,
        size=3.0,
        alpha=0.5,
        edgecolor="#2b2b2b",
        linewidth=0.4,
        marker="o",
        ax=ax,
        legend=False,
    )
    ax.set_xlabel("Model")
    ax.set_ylabel("Per-gene Pearson")
    ax.set_title(f"{dataset} | Split Comparison Overview | {title_suffix}")
    plt.setp(ax.get_xticklabels(), rotation=35, ha="right")
    ax.legend(title="Split", bbox_to_anchor=(1.02, 1), loc="upper left")
    sns.despine(fig_split, left=True, bottom=True)
    fig_split.tight_layout()
    register_figure(FIGURES, f"split_comparison_overview_{dataset.lower()}", fig_split)
    display(fig_split)
    plt.close(fig_split)

    # Boxplots (half violin + half box + scatter) ordered by test Pearson
    def _half_violin_box(metric_key: str, title_label: str) -> None:
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.violinplot(
            data=ds_test,
            x="model_display",
            y=metric_key,
            order=model_order,
            ax=ax,
            palette="Set2",
            inner=None,
            cut=0,
            linewidth=1.0,
        )
        for idx, poly in enumerate(ax.collections[:len(model_order)]):
            for path in poly.get_paths():
                verts = path.vertices
                verts[:, 0] = np.clip(verts[:, 0], idx, np.inf)

        data_by_model = [
            ds_test.loc[ds_test["model_display"] == name, metric_key].dropna().values
            for name in model_order
        ]
        positions = np.arange(len(model_order)) - 0.2
        box = ax.boxplot(
            data_by_model,
            positions=positions,
            widths=0.2,
            patch_artist=True,
            showfliers=False,
            medianprops={"color": "#222222", "linewidth": 1.0},
        )
        for patch in box["boxes"]:
            patch.set(facecolor="#ffffff", edgecolor="#222222", linewidth=1.0)
        for whisker in box["whiskers"]:
            whisker.set(color="#222222", linewidth=1.0)
        for cap in box["caps"]:
            cap.set(color="#222222", linewidth=1.0)

        rng_local = np.random.default_rng(config.random_seed)
        for idx, vals in enumerate(data_by_model):
            if len(vals) == 0:
                continue
            jitter = rng_local.normal(0, 0.03, size=len(vals))
            x_vals = np.full(len(vals), positions[idx]) + jitter
            ax.scatter(x_vals, vals, s=10, alpha=0.35, color="#2b2b2b", linewidth=0)

        ax.set_title(f"{dataset} | Test {title_label} by Model | {title_suffix}")
        ax.set_xlabel("Model")
        ax.set_ylabel(title_label)
        ax.set_xticks(np.arange(len(model_order)))
        ax.set_xticklabels(model_order, rotation=45, ha="right")
        ax.set_xlim(-0.6, len(model_order) - 0.4)
        sns.despine(ax=ax, left=True, bottom=True)
        plt.tight_layout()
        register_figure(FIGURES, f"test_{metric_key}_boxplot_{dataset.lower()}", fig)
        display(fig)
        plt.close(fig)

    for metric_key, title_label in metrics_to_plot.items():
        if metric_key in ds_test.columns:
            _half_violin_box(metric_key, title_label)

    # Combined mean-metric heatmap (per dataset)
    metric_means = {}
    for metric_key, title_label in metrics_to_plot.items():
        if metric_key not in ds_test.columns:
            continue
        series = ds_test.groupby("model_display")[metric_key].mean().dropna()
        if not series.empty:
            metric_means[title_label] = series

    if metric_means:
        combined_df = pd.DataFrame(metric_means).reindex(index=model_order)
        fig_height = max(4, 0.4 * len(combined_df.index))
        metric_cols = list(combined_df.columns)
        fig, axes = plt.subplots(
            nrows=1,
            ncols=len(metric_cols),
            figsize=(1.6 * len(metric_cols), fig_height),
            squeeze=False,
        )
        for idx, metric in enumerate(metric_cols):
            ax = axes[0][idx]
            col_data = combined_df[[metric]]
            vmin_col, vmax_col = compute_heatmap_limits(col_data.values, lower_percentile=10.0, upper_percentile=95.0)
            sns.heatmap(
                col_data,
                cmap="viridis",
                vmin=vmin_col,
                vmax=vmax_col,
                annot=True,
                fmt=".3f",
                linewidths=0.2,
                linecolor="#f5f5f5",
                cbar=False,
                ax=ax,
            )
            ax.set_title("")
            ax.set_xlabel("")
            ax.set_ylabel("Model" if idx == 0 else "")
            if idx == 0:
                ax.yaxis.labelpad = 20
            ax.set_yticklabels(ax.get_yticklabels(), fontsize=8)
            if idx != 0:
                ax.set_yticklabels([])
        fig.suptitle(f"{dataset} | Summary metrics (test means) | {title_suffix}", fontsize=12)
        sns.despine(fig, left=True, bottom=True)
        plt.tight_layout()
        register_figure(FIGURES, f"test_metric_mean_heatmap_combined_{dataset.lower()}", fig)
        display(fig)
        plt.close(fig)

    # Top-20 per-model gene blocks (per dataset)
    if "pearson" in ds_test.columns:
        test_pivot = ds_test.pivot_table(
            index="gene",
            columns="model_display",
            values="pearson",
            aggfunc="mean",
        )
        values = {}
        annot = {}
        for model_name in model_order:
            if model_name not in test_pivot.columns:
                continue
            series = test_pivot[model_name].dropna().sort_values(ascending=False).head(20)
            padded_values = series.tolist() + [float("nan")] * (20 - len(series))
            padded_genes = series.index.tolist() + [""] * (20 - len(series))
            values[model_name] = padded_values
            annot[model_name] = ["{}\n{:.2f}".format(g, v) if g else "" for g, v in zip(padded_genes, padded_values)]

        if values:
            heatmap_df = pd.DataFrame(values, index=[f"rank {i}" for i in range(1, 21)])
            annot_df = pd.DataFrame(annot, index=heatmap_df.index)
            fig_width = max(8, 1.2 * len(heatmap_df.columns))
            fig_height = 10
            fig_blocks, ax = plt.subplots(figsize=(fig_width, fig_height))
            sns.heatmap(
                heatmap_df,
                cmap="crest",
                annot=annot_df,
                fmt="",
                linewidths=0.3,
                linecolor="#f5f5f5",
                cbar_kws={"label": "Test Pearson"},
                ax=ax,
            )
            ax.set_title(f"{dataset} | Test Pearson (top 20 genes per model) | {title_suffix}")
            ax.set_xlabel("Model")
            ax.set_ylabel("Rank (per model)")
            ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", fontsize=9)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize=9)
            sns.despine(fig_blocks, left=True, bottom=True)
            plt.tight_layout()
            register_figure(FIGURES, f"test_pearson_heatmap_top20_per_model_blocks_{dataset.lower()}", fig_blocks)
            display(fig_blocks)
            plt.close(fig_blocks)


In [ ]:
summary_reset = analysis_state["summary_reset"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)

title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"

fig_gap = None
required_cols = {"train_pearson_mean", "test_pearson_mean"}
subset = summary_reset[summary_reset["dataset"].isin(["Embryonic", "Endothelial"])].copy()
if subset.empty or not required_cols.issubset(subset.columns):
    print("Train/test summary columns unavailable; skipping generalization gap plot.")
else:
    fig_gap, axes = plt.subplots(ncols=2, figsize=(12, 6), sharex=True)
    for ax, dataset in zip(axes, ["Embryonic", "Endothelial"]):
        data = subset[subset["dataset"] == dataset].copy()
        if data.empty:
            ax.axis("off")
            ax.set_title(f"{dataset} (no data)")
            continue
        ranked = data.sort_values("test_pearson_mean", ascending=False)
        best_per_model = ranked.drop_duplicates("model_id")
        gap_df = best_per_model[["model_display", "model_id", "train_pearson_mean", "test_pearson_mean"]].copy()
        gap_df["generalization_gap"] = gap_df["train_pearson_mean"] - gap_df["test_pearson_mean"]
        # Sort ascending=True so low gaps are at top, high gaps at bottom
        gap_df.sort_values("generalization_gap", ascending=True, inplace=True)
        colors = sns.color_palette("mako", n_colors=len(gap_df))
        bars = ax.barh(
            gap_df["model_display"],
            gap_df["generalization_gap"],
            color=colors,
            linewidth=0,
        )
        ax.axvline(0.0, color="#6d6d6d", linestyle="--", linewidth=1)
        gap_min = gap_df["generalization_gap"].min()
        gap_max = gap_df["generalization_gap"].max()
        span = max(0.01, gap_max - gap_min)
        margin = max(0.08, span * 0.12)
        xmin = min(gap_min - margin, gap_min - 0.02)
        xmin = min(xmin, -0.15)
        xmax = max(gap_max + margin, 0.15)
        ax.set_xlim(xmin, xmax)
        ax.set_xlabel("Train - Test Mean Pearson")
        ax.set_ylabel("Model")
        ax.set_title(dataset)
        ax.invert_yaxis()
        label_offset = (xmax - xmin) * 0.02
        for bar, (_, row) in zip(bars, gap_df.iterrows()):
            y = bar.get_y() + bar.get_height() / 2
            value = row["generalization_gap"]
            if value >= 0:
                ax.text(value + label_offset, y, f"{value:.3f}", va="center", ha="left", fontsize=9, color="#1f1f1f")
            else:
                ax.text(value - label_offset, y, f"{value:.3f}", va="center", ha="right", fontsize=9, color="#1f1f1f")
        sns.despine(ax=ax, left=True, bottom=True)
    fig_gap.suptitle(f"Generalization Gap (Train - Test) | {title_suffix}", fontsize=12)
    fig_gap.tight_layout(rect=[0, 0, 1, 0.92])

register_figure(FIGURES, "generalization_gap_by_dataset", fig_gap)
if fig_gap is not None:
    display(fig_gap)
    plt.close(fig_gap)

## 10. Split Comparison by Dataset


In [ ]:
metrics_long = analysis_state["metrics_long"].copy()
metrics_long["dataset"] = metrics_long["run_name"].map(_assign_dataset)
model_display_order = analysis_state["model_display_order"]

splits_of_interest = [config.train_split, config.val_split, config.primary_split]
subset = metrics_long[
    metrics_long["split"].isin(splits_of_interest)
    & metrics_long["dataset"].isin(["Embryonic", "Endothelial"])
].copy()

fig_split_compare = None
if subset.empty or "pearson" not in subset:
    print("Pearson metrics unavailable across requested splits; skipping split comparison plot.")
else:
    subset = subset[["model_display", "split", "pearson", "dataset"]].dropna()
    split_labels = {
        config.train_split: "Train",
        config.val_split: "Val",
        config.primary_split: "Test",
    }
    subset["split_label"] = subset["split"].map(split_labels).fillna(subset["split"].str.title())

    ordered_models = [model for model in model_display_order if model in subset["model_display"].unique()]
    if not ordered_models:
        ordered_models = subset["model_display"].sort_values().unique().tolist()

    # Build 6 hues per model: 3 splits per dataset with shade ramps
    hue_order = [
        "Embryonic | Train",
        "Embryonic | Val",
        "Embryonic | Test",
        "Endothelial | Train",
        "Endothelial | Val",
        "Endothelial | Test",
    ]
    palette = {
        "Embryonic | Train": "#9ecae1",
        "Embryonic | Val": "#6baed6",
        "Embryonic | Test": "#3182bd",
        "Endothelial | Train": "#fdd0a2",
        "Endothelial | Val": "#fdae6b",
        "Endothelial | Test": "#e6550d",
    }
    subset["dataset_split"] = subset["dataset"] + " | " + subset["split_label"]

    fig_width = max(16, 1.2 * max(8, len(ordered_models)))
    fig_split_compare, ax = plt.subplots(figsize=(fig_width, 7.2))
    sns.boxplot(
        data=subset,
        x="model_display",
        y="pearson",
        hue="dataset_split",
        order=ordered_models,
        hue_order=hue_order,
        palette=palette,
        width=0.7,
        fliersize=0,
        ax=ax,
    )
    jitter_sample = subset.sample(min(len(subset), 5000), random_state=config.random_seed) if len(subset) > 5000 else subset
    sns.stripplot(
        data=jitter_sample,
        x="model_display",
        y="pearson",
        hue="dataset_split",
        order=ordered_models,
        hue_order=hue_order,
        palette=palette,
        dodge=True,
        jitter=0.12,
        size=2.4,
        alpha=0.35,
        edgecolor="#2b2b2b",
        linewidth=0.3,
        marker="o",
        ax=ax,
        legend=False,
    )
    metric_min = subset["pearson"].min()
    metric_max = subset["pearson"].max()
    ymin = min(-0.5, metric_min - 0.05)
    ymax = max(1.0, metric_max + 0.05)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel("Model")
    ax.set_ylabel("Per-gene Pearson")
    ax.set_title("Per-gene Pearson by Split and Dataset")
    plt.setp(ax.get_xticklabels(), rotation=35, ha="right")
    ax.legend(title="Dataset | Split", bbox_to_anchor=(1.02, 1), loc="upper left")
    sns.despine(fig_split_compare, left=True, bottom=True)
    fig_split_compare.tight_layout()

register_figure(FIGURES, "split_comparison_by_dataset", fig_split_compare)
if fig_split_compare is not None:
    display(fig_split_compare)
    plt.close(fig_split_compare)


## 11. Best Model Diagnostics per Dataset


In [ ]:
def _load_feature_importance_table(model_dir: Path) -> tuple[pd.DataFrame, Optional[Path]]:
    # Locate and standardise feature importances exported by the training pipeline.
    patterns = [
        "feature_importance*.csv",
        "feature_importances*.csv",
        "feature_importance*.tsv",
        "feature_importances*.tsv",
        "feature_importance*.parquet",
        "feature_importances*.parquet",
    ]
    candidates: list[Path] = []
    for pattern in patterns:
        candidates.extend(model_dir.glob(pattern))
    if not candidates:
        for pattern in patterns:
            candidates.extend(model_dir.glob(f"**/{pattern}"))
    unique_candidates: list[Path] = []
    seen: set[Path] = set()
    for path in candidates:
        resolved = path.resolve()
        if resolved in seen:
            continue
        if resolved.suffix.lower() in {".png", ".jpg", ".jpeg"}:
            continue
        seen.add(resolved)
        unique_candidates.append(resolved)
    for candidate in sorted(unique_candidates):
        try:
            if candidate.suffix.lower() == ".parquet":
                df = pd.read_parquet(candidate)
            else:
                sep = "	" if candidate.suffix.lower() in {".tsv", ".txt"} else ","
                df = pd.read_csv(candidate, sep=sep)
        except Exception as exc:
            print(f"Skipping {candidate.name}: failed to load ({exc})")
            continue
        if df.empty:
            continue
        lower_cols = {col.lower(): col for col in df.columns}
        feature_col = next((
            lower_cols[key]
            for key in ("feature", "feature_name", "name", "variable", "feature_id", "column")
            if key in lower_cols
        ), None)
        importance_col = next((
            lower_cols[key]
            for key in (
                "importance",
                "importance_score",
                "importance_mean",
                "score",
                "value",
                "gain",
                "weight",
            )
            if key in lower_cols
        ), None)
        if feature_col is None or importance_col is None:
            continue
        out = df.copy()
        out.rename(columns={feature_col: "feature", importance_col: "importance"}, inplace=True)
        out["feature"] = out["feature"].astype(str)
        out["importance"] = pd.to_numeric(out["importance"], errors="coerce")
        out = out.dropna(subset=["feature", "importance"])
        if out.empty:
            continue
        extra_cols = [col for col in out.columns if col not in {"feature", "importance"}]
        out = out[["feature", "importance", *extra_cols]]
        out.sort_values("importance", ascending=False, inplace=True)
        return out.reset_index(drop=True), candidate
    return pd.DataFrame(columns=["feature", "importance"]), None


def _best_model_for_dataset(summary_reset: pd.DataFrame, dataset: str) -> Optional[pd.Series]:
    subset = summary_reset[summary_reset["dataset"] == dataset].copy()
    if subset.empty:
        return None
    if "test_pearson_mean" in subset:
        subset.sort_values("test_pearson_mean", ascending=False, inplace=True)
    return subset.iloc[0]


def _ensure_distance_column(df: pd.DataFrame) -> tuple[pd.DataFrame, Optional[str], bool]:
    if df is None or df.empty:
        return df, None, False

    signed_candidates = [
        ("delta_to_tss_kb", True),
        ("distance_to_tss_kb", True),
        ("delta_to_tss_bp", False),
        ("distance_to_tss_bp", False),
    ]
    for name, is_kb in signed_candidates:
        if name in df.columns:
            values = pd.to_numeric(df[name], errors="coerce")
            if (values.dropna() < 0).any():
                return df, name, is_kb

    def _infer_signed_distance(series: pd.Series) -> pd.Series:
        import re
        bin_pattern = re.compile(r"bin_(-?\d+)_to_(-?\d+)")
        def _infer_distance(feature_name: str) -> Optional[float]:
            if not isinstance(feature_name, str):
                return None
            token = feature_name.split("|", 1)[-1]
            match = bin_pattern.search(token)
            if match:
                start_bp = float(match.group(1))
                end_bp = float(match.group(2))
                return 0.5 * (start_bp + end_bp)
            return None
        return series.map(_infer_distance)

    if "feature" in df.columns:
        inferred = _infer_signed_distance(df["feature"])
        if inferred.notna().any():
            df = df.copy()
            df["distance_to_tss_bp_signed"] = inferred
            return df, "distance_to_tss_bp_signed", False

    return df, None, False


def _plot_feature_importance_distance(
    df: pd.DataFrame,
    distance_col: str,
    distance_is_kb: bool,
    title: str,
    *,
    max_distance_kb: float = 10.0,
    y_limits: Optional[tuple[float, float]] = None,
    show_scatter: bool = False,
) -> Optional[plt.Figure]:
    if df is None or df.empty or distance_col is None:
        return None
    valid = df.dropna(subset=["importance", distance_col]).copy()
    if valid.empty:
        return None
    distance_values = pd.to_numeric(valid[distance_col], errors="coerce")
    valid["distance_kb"] = distance_values if distance_is_kb else distance_values / 1_000.0
    valid = valid[valid["distance_kb"].abs() <= max_distance_kb].copy()
    if valid.empty:
        return None
    if not (valid["distance_kb"] < 0).any():
        print("Warning: no negative distance-to-TSS values detected; data may be absolute distances.")
    valid.sort_values("distance_kb", inplace=True)

    per_bin = (
        valid.groupby("distance_kb", sort=True)["importance"]
        .quantile(0.9)
        .reset_index()
    )
    if per_bin.empty:
        return None

    fig, ax = plt.subplots(figsize=(7.5, 4.8))
    if show_scatter:
        sns.scatterplot(
            data=valid,
            x="distance_kb",
            y="importance",
            s=30,
            alpha=0.45,
            edgecolor="none",
            color="#4daf4a",
            ax=ax,
        )
    ax.plot(
        per_bin["distance_kb"],
        per_bin["importance"],
        color="#e41a1c",
        linewidth=1.5,
        label="90th percentile",
    )
    ax.legend(loc="upper right", frameon=True)
    ax.axvline(0.0, color="#999999", linestyle="--", linewidth=1)
    ax.set_xlim(-max_distance_kb, max_distance_kb)
    if y_limits is not None:
        ax.set_ylim(y_limits)
    ax.set_xlabel("Distance to TSS (kb)")
    ax.set_ylabel("Feature importance")
    ax.set_title(f"{title} (±{max_distance_kb:g} kb)")
    sns.despine(fig, left=True, bottom=True)
    plt.tight_layout()
    return fig


summary_reset = analysis_state["summary_reset"].copy()
title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)
run_df = analysis_state["run_df"]

for dataset in ["Embryonic", "Endothelial"]:
    best_row = _best_model_for_dataset(summary_reset, dataset)
    if best_row is None:
        print(f"No best model found for {dataset}.")
        continue
    run_match = run_df[
        (run_df["run_name"] == best_row["run_name"]) & (run_df["model_id"] == best_row["model_id"])
    ]
    if run_match.empty:
        print(f"Run metadata missing for {dataset} best model.")
        continue
    row = run_match.iloc[0]
    model_dir = row["model_path"] if isinstance(row["model_path"], Path) else Path(row["model_path"])
    scatter_path = model_dir / "scatter_test.png"
    display(Markdown(
        f"**{dataset} best model:** `{best_row['model_display']}` (run `{best_row['run_name']}`)"
    ))
    if scatter_path.exists():
        display(Image(filename=str(scatter_path)))
        EXTRA_EXPORT_PATHS.append((scatter_path, f"{dataset.lower()}_scatter_test.png"))
    else:
        preds_path = row.get("predictions_path")
        preds_path = preds_path if isinstance(preds_path, Path) else Path(preds_path) if preds_path else None
        if preds_path is None or not preds_path.exists():
            print(f"No scatter or prediction file found for {dataset} best model.")
        else:
            df = pd.read_csv(preds_path)
            df = df[df["split"] == config.primary_split]
            if df.empty:
                print(f"No test predictions available for {dataset} best model.")
            else:
                sample = df.sample(min(len(df), 8000), random_state=config.random_seed)
                fig_scatter, ax = plt.subplots(figsize=(5.2, 5.2))
                sns.scatterplot(
                    data=sample,
                    x="y_true",
                    y="y_pred",
                    s=14,
                    alpha=0.4,
                    edgecolor="none",
                    color="#377eb8",
                    ax=ax,
                )
                lims = [
                    min(sample["y_true"].min(), sample["y_pred"].min()),
                    max(sample["y_true"].max(), sample["y_pred"].max()),
                ]
                ax.plot(lims, lims, linestyle="--", color="#6d6d6d", linewidth=1)
                ax.set_xlim(lims)
                ax.set_ylim(lims)
                ax.set_xlabel("Observed")
                ax.set_ylabel("Predicted")
                ax.set_title(f"{dataset}: Test Scatter | {best_row['model_display']} | {title_suffix}")
                sns.despine(fig_scatter, left=True, bottom=True)
                plt.tight_layout()
                register_figure(FIGURES, f"{dataset.lower()}_best_model_scatter", fig_scatter)
                display(fig_scatter)
                plt.close(fig_scatter)

    importance_df, importance_path = _load_feature_importance_table(model_dir)
    if importance_df.empty:
        print(f"No feature importance table found for {dataset} best model.")
        continue
    importance_df, distance_col, distance_is_kb = _ensure_distance_column(importance_df)
    if distance_col is None or distance_col not in importance_df.columns:
        print(f"Unable to infer distance-to-TSS column for {dataset} best model.")
        continue
    fig_distance = _plot_feature_importance_distance(
        importance_df,
        distance_col,
        distance_is_kb,
        f"{dataset}: Feature Importance vs Distance to TSS | {title_suffix}",
        max_distance_kb=10.0,
        show_scatter=True,
    )
    zoom_mask = importance_df[distance_col].notna()
    if zoom_mask.any():
        distance_values = pd.to_numeric(importance_df.loc[zoom_mask, distance_col], errors="coerce")
        distance_kb = distance_values if distance_is_kb else distance_values / 1_000.0
        zoom_range = distance_kb.abs() <= 5.0
        zoom_importance = pd.to_numeric(importance_df.loc[zoom_mask, "importance"], errors="coerce")
        zoom_df = pd.DataFrame({"distance_kb": distance_kb[zoom_range], "importance": zoom_importance[zoom_range]})
        if zoom_df.empty:
            zoom_max = None
        else:
            zoom_max = zoom_df.groupby("distance_kb")["importance"].quantile(0.9).max()
        zoom_y_limits = (0.0, float(zoom_max) * 1.15) if pd.notna(zoom_max) else None
    else:
        zoom_y_limits = None
    fig_distance_zoomed = _plot_feature_importance_distance(
        importance_df,
        distance_col,
        distance_is_kb,
        f"{dataset}: Feature Importance vs Distance to TSS (zoomed) | {title_suffix}",
        max_distance_kb=5.0,
        y_limits=zoom_y_limits,
    )
    if fig_distance is None or fig_distance_zoomed is None:
        print(f"No valid distance/importance data for {dataset} best model.")
    else:
        model_short = MODEL_ID_TO_SHORT.get(best_row['model_id'], best_row['model_display'])
        register_figure(FIGURES, f"{dataset.lower()}_{model_short}_feature_importance_tss", fig_distance)
        display(fig_distance)
        plt.close(fig_distance)
        register_figure(FIGURES, f"{dataset.lower()}_{model_short}_feature_importance_tss_zoomed", fig_distance_zoomed)
        display(fig_distance_zoomed)
        plt.close(fig_distance_zoomed)


## 12. Best Model SHAP Values per Dataset

SHAP (SHapley Additive exPlanations) values provide model-agnostic feature importance.
If SHAP values were computed during training (enable_shap=true), plot them vs TSS distance.


In [ ]:
def _load_shap_table(model_dir: Path) -> tuple[pd.DataFrame, Optional[Path]]:
    """Load SHAP importance table from model directory."""
    patterns = [
        "shap_importance*.csv",
        "shap_importances*.csv",
    ]
    candidates: list[Path] = []
    for pattern in patterns:
        candidates.extend(model_dir.glob(pattern))
    if not candidates:
        for pattern in patterns:
            candidates.extend(model_dir.glob(f"**/{pattern}"))
    
    unique_candidates: list[Path] = []
    seen: set[Path] = set()
    for path in candidates:
        resolved = path.resolve()
        if resolved in seen or resolved.suffix.lower() in {".png", ".jpg", ".jpeg"}:
            continue
        seen.add(resolved)
        unique_candidates.append(resolved)
    
    for candidate in sorted(unique_candidates):
        try:
            df = pd.read_csv(candidate)
        except Exception as exc:
            print(f"Skipping {candidate.name}: failed to load ({exc})")
            continue
        if df.empty:
            continue
        
        # Check for required columns
        lower_cols = {col.lower(): col for col in df.columns}
        feature_col = next((
            lower_cols[key]
            for key in ("feature", "feature_name", "name")
            if key in lower_cols
        ), None)
        shap_col = next((
            lower_cols[key]
            for key in ("shap_mean_abs", "shap_value", "shap", "shap_importance")
            if key in lower_cols
        ), None)
        
        if feature_col is None or shap_col is None:
            continue
        
        out = df.copy()
        out.rename(columns={feature_col: "feature", shap_col: "shap_value"}, inplace=True)
        out["feature"] = out["feature"].astype(str)
        out["shap_value"] = pd.to_numeric(out["shap_value"], errors="coerce")
        out = out.dropna(subset=["feature", "shap_value"])
        if out.empty:
            continue
        
        extra_cols = [col for col in out.columns if col not in {"feature", "shap_value"}]
        out = out[["feature", "shap_value", *extra_cols]]
        out.sort_values("shap_value", ascending=False, inplace=True)
        return out.reset_index(drop=True), candidate
    
    return pd.DataFrame(columns=["feature", "shap_value"]), None


def _plot_shap_vs_tss_distance(
    df: pd.DataFrame,
    distance_col: str,
    distance_is_kb: bool,
    title: str,
    *,
    max_distance_kb: float = 10.0,
    y_limits: Optional[tuple[float, float]] = None,
    show_scatter: bool = False,
) -> Optional[plt.Figure]:
    """Plot SHAP values vs TSS distance in manuscript style."""
    if df is None or df.empty or distance_col is None:
        return None
    valid = df.dropna(subset=["shap_value", distance_col]).copy()
    if valid.empty:
        return None
    distance_values = pd.to_numeric(valid[distance_col], errors="coerce")
    valid["distance_kb"] = distance_values if distance_is_kb else distance_values / 1_000.0
    valid = valid[valid["distance_kb"].abs() <= max_distance_kb].copy()
    if valid.empty:
        return None
    if not (valid["distance_kb"] < 0).any():
        print("Warning: no negative distance-to-TSS values detected; data may be absolute distances.")
    valid.sort_values("distance_kb", inplace=True)

    per_bin = (
        valid.groupby("distance_kb", sort=True)["shap_value"]
        .quantile(0.9)
        .reset_index()
    )
    if per_bin.empty:
        return None

    fig, ax = plt.subplots(figsize=(7.5, 4.8))
    if show_scatter:
        sns.scatterplot(
            data=valid,
            x="distance_kb",
            y="shap_value",
            s=30,
            alpha=0.45,
            edgecolor="none",
            color="#4daf4a",
            ax=ax,
        )
    ax.plot(
        per_bin["distance_kb"],
        per_bin["shap_value"],
        color="#e41a1c",
        linewidth=1.5,
        label="90th percentile",
    )
    ax.legend(loc="upper right", frameon=True)
    ax.axvline(0.0, color="#999999", linestyle="--", linewidth=1)
    ax.set_xlim(-max_distance_kb, max_distance_kb)
    if y_limits is not None:
        ax.set_ylim(y_limits)
    ax.set_xlabel("Distance to TSS (kb)")
    ax.set_ylabel("SHAP value")
    ax.set_title(f"{title} (±{max_distance_kb:g} kb)")
    sns.despine(fig, left=True, bottom=True)
    plt.tight_layout()
    return fig


# Plot SHAP values vs TSS distance for best models
summary_reset = analysis_state["summary_reset"].copy()
title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)
run_df = analysis_state["run_df"]

shap_plots_created = False
for dataset in ["Embryonic", "Endothelial"]:
    best_row = _best_model_for_dataset(summary_reset, dataset)
    if best_row is None:
        continue
    run_match = run_df[
        (run_df["run_name"] == best_row["run_name"]) & (run_df["model_id"] == best_row["model_id"])
    ]
    if run_match.empty:
        continue
    row = run_match.iloc[0]
    model_dir = row["model_path"] if isinstance(row["model_path"], Path) else Path(row["model_path"])
    
    shap_df, shap_path = _load_shap_table(model_dir)
    if shap_df.empty:
        continue
    
    display(Markdown(
        f"**{dataset} SHAP values:** `{best_row['model_display']}` (run `{best_row['run_name']}`)"
    ))
    
    shap_df, distance_col, distance_is_kb = _ensure_distance_column(shap_df)
    if distance_col is None or distance_col not in shap_df.columns:
        print(f"Unable to infer distance-to-TSS column for {dataset} SHAP values.")
        continue
    
    fig_shap = _plot_shap_vs_tss_distance(
        shap_df,
        distance_col,
        distance_is_kb,
        f"{dataset}: SHAP Values vs Distance to TSS | {title_suffix}",
        max_distance_kb=10.0,
        show_scatter=True,
    )
    
    # Zoomed version (±5kb)
    zoom_mask = shap_df[distance_col].notna()
    if zoom_mask.any():
        distance_values = pd.to_numeric(shap_df.loc[zoom_mask, distance_col], errors="coerce")
        distance_kb = distance_values if distance_is_kb else distance_values / 1_000.0
        zoom_range = distance_kb.abs() <= 5.0
        zoom_shap = pd.to_numeric(shap_df.loc[zoom_mask, "shap_value"], errors="coerce")
        zoom_df = pd.DataFrame({"distance_kb": distance_kb[zoom_range], "shap_value": zoom_shap[zoom_range]})
        if zoom_df.empty:
            zoom_max = None
        else:
            zoom_max = zoom_df.groupby("distance_kb")["shap_value"].quantile(0.9).max()
        zoom_y_limits = (0.0, float(zoom_max) * 1.15) if pd.notna(zoom_max) else None
    else:
        zoom_y_limits = None
    
    fig_shap_zoomed = _plot_shap_vs_tss_distance(
        shap_df,
        distance_col,
        distance_is_kb,
        f"{dataset}: SHAP Values vs Distance to TSS (zoomed) | {title_suffix}",
        max_distance_kb=5.0,
        y_limits=zoom_y_limits,
    )
    
    if fig_shap is None or fig_shap_zoomed is None:
        print(f"No valid distance/SHAP data for {dataset} best model.")
    else:
        model_short = MODEL_ID_TO_SHORT.get(best_row['model_id'], best_row['model_display'])
        register_figure(FIGURES, f"{dataset.lower()}_{model_short}_shap_tss", fig_shap)
        display(fig_shap)
        plt.close(fig_shap)
        register_figure(FIGURES, f"{dataset.lower()}_{model_short}_shap_tss_zoomed", fig_shap_zoomed)
        display(fig_shap_zoomed)
        plt.close(fig_shap_zoomed)
        shap_plots_created = True

if not shap_plots_created:
    print("No SHAP values found. Ensure models were trained with enable_shap=true.")

In [ ]:
run_df = analysis_state["run_df"]
summary_reset = analysis_state["summary_reset"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)

# Plot training curves for best models
fig_training_curves = None
training_data_found = False

for dataset in ["Embryonic", "Endothelial"]:
    best_row = _best_model_for_dataset(summary_reset, dataset)
    if best_row is None:
        continue
    run_match = run_df[
        (run_df["run_name"] == best_row["run_name"]) & (run_df["model_id"] == best_row["model_id"])
    ]
    if run_match.empty:
        continue
    row = run_match.iloc[0]
    history_path = row.get("training_history_path")
    history_path = history_path if isinstance(history_path, Path) else Path(history_path) if history_path else None
    
    if history_path is None or not history_path.exists():
        continue
    
    training_data_found = True
    try:
        history_df = pd.read_csv(history_path)
        if history_df.empty or not {"loss", "epoch"}.issubset(history_df.columns):
            continue
        
        if fig_training_curves is None:
            fig_training_curves, axes = plt.subplots(ncols=2, figsize=(13, 5))
        
        ax = axes[0] if dataset == "Embryonic" else axes[1]
        
        # Plot loss curves
        if "loss" in history_df:
            ax.plot(history_df["epoch"], history_df["loss"], label="Train Loss", linewidth=2, color="#1f77b4")
        if "val_loss" in history_df:
            ax.plot(history_df["epoch"], history_df["val_loss"], label="Val Loss", linewidth=2, color="#ff7f0e")
        
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.set_title(f"{dataset}: {best_row['model_display']} Training")
        ax.legend()
        ax.grid(True, alpha=0.3)
        sns.despine(ax=ax, left=True, bottom=True)
    except Exception as e:
        print(f"Failed to load training history for {dataset}: {e}")

if fig_training_curves is not None:
    fig_training_curves.tight_layout()
    register_figure(FIGURES, "training_curves", fig_training_curves)
    display(fig_training_curves)
    plt.close(fig_training_curves)
elif training_data_found:
    print("Training history available but could not generate combined plot.")
else:
    print("No training history files found in model outputs.")

## 13. Top/Bottom Genes per Dataset


In [ ]:
metrics_wide = analysis_state["metrics_wide"].copy()
summary_reset = analysis_state["summary_reset"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)

# Top and bottom performing genes - SPLIT BY DATASET
if not metrics_wide.empty and f"{config.primary_split}_pearson" in metrics_wide:
    metrics_wide_with_dataset = metrics_wide.copy()
    metrics_wide_with_dataset["dataset"] = metrics_wide_with_dataset["run_name"].map(_assign_dataset)
    
    for dataset in ["Embryonic", "Endothelial"]:
        dataset_metrics = metrics_wide_with_dataset[
            metrics_wide_with_dataset["dataset"] == dataset
        ].copy()
        
        if dataset_metrics.empty:
            print(f"No data available for {dataset} gene extremes.")
            continue
        
        gene_stats = dataset_metrics.groupby("gene")[f"{config.primary_split}_pearson"].agg(["mean", "std", "count"])
        gene_stats = gene_stats.sort_values("mean", ascending=False)

        n_top = min(20, len(gene_stats) // 2)
        if n_top > 0:
            top_genes = gene_stats.head(n_top)
            bottom_genes = gene_stats.tail(n_top)
            extremes = pd.concat([top_genes, bottom_genes])
            extremes["category"] = ["Top"] * len(top_genes) + ["Bottom"] * len(bottom_genes)

            fig_gene_extremes, ax = plt.subplots(figsize=(11, 8))
            colors = ["#2ecc71" if c == "Top" else "#e74c3c" for c in extremes["category"]]

            y_pos = np.arange(len(extremes))
            ax.barh(y_pos, extremes["mean"], xerr=extremes["std"], color=colors, capsize=4, alpha=0.85)

            ax.set_yticks(y_pos)
            ax.set_yticklabels(extremes.index, fontsize=9)
            ax.set_xlabel("Mean Test Pearson (±std)")
            ax.set_title(f"{dataset}: Top and Bottom 20 Genes Across All Models")
            ax.invert_yaxis()
            sns.despine(fig_gene_extremes, left=True, bottom=True)
            fig_gene_extremes.tight_layout()
            register_figure(FIGURES, f"gene_extremes_{dataset.lower()}", fig_gene_extremes)
            display(fig_gene_extremes)
            plt.close(fig_gene_extremes)
        else:
            print(f"Insufficient genes for {dataset} extremes plot.")
else:
    print("Test Pearson per-gene data unavailable for gene extremes.")

# Prediction error distribution by gene (best model per dataset) - SPLIT BY DATASET
if not metrics_wide.empty and f"{config.primary_split}_pearson" in metrics_wide:
    for dataset in ["Embryonic", "Endothelial"]:
        subset = summary_reset[summary_reset["dataset"] == dataset].copy()
        if subset.empty:
            print(f"{dataset}: no data available for gene pearson distribution")
            continue
        subset = subset.sort_values("test_pearson_mean", ascending=False)
        best_row = subset.iloc[0]
        model_mask = (
            (metrics_wide["run_name"] == best_row["run_name"])
            & (metrics_wide["model_id"] == best_row["model_id"])
        )
        best_metrics = metrics_wide[model_mask]
        if best_metrics.empty:
            print(f"{dataset}: missing metrics for best model")
            continue
        gene_pearson = best_metrics.groupby("gene")[f"{config.primary_split}_pearson"].mean()
        
        fig_pred_error, ax = plt.subplots(figsize=(8, 5))
        ax.hist(gene_pearson, bins=max(10, len(gene_pearson) // 3), color="#3498db", alpha=0.7, edgecolor="black")
        ax.axvline(gene_pearson.mean(), color="red", linestyle="--", linewidth=2, label=f"Mean: {gene_pearson.mean():.3f}")
        ax.axvline(gene_pearson.median(), color="orange", linestyle="--", linewidth=2, label=f"Median: {gene_pearson.median():.3f}")
        model_label = best_row["model_display"]
        ax.set_title(f"{dataset} Best Model: Per-Gene Test Pearson Distribution\n({model_label})")
        ax.set_xlabel("Mean Test Pearson per Gene")
        ax.set_ylabel("Frequency")
        ax.legend(fontsize=9)
        sns.despine(fig_pred_error, left=True, bottom=True)
        fig_pred_error.tight_layout()
        register_figure(FIGURES, f"gene_pearson_distribution_{dataset.lower()}", fig_pred_error)
        display(fig_pred_error)
        plt.close(fig_pred_error)
else:
    print("Test Pearson per-gene data unavailable for gene pearson distribution.")

## 14. Cross-Dataset Performance Comparison


In [ ]:
# Cross-dataset performance comparison
summary_reset = analysis_state["summary_reset"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)

fig_cross_dataset = None
title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"
if not summary_reset.empty and "test_pearson_mean" in summary_reset:
    cross_perf_mean = summary_reset.pivot_table(
        index="model_display",
        columns="dataset",
        values="test_pearson_mean",
        aggfunc="mean",
    )
    cross_perf_std = summary_reset.pivot_table(
        index="model_display",
        columns="dataset",
        values="test_pearson_mean",
        aggfunc="std",
    )

    if not cross_perf_mean.empty and len(cross_perf_mean.columns) > 1:
        cross_perf_mean = cross_perf_mean.dropna(how="any")
        cross_perf_std = cross_perf_std.reindex(cross_perf_mean.index)

        if not cross_perf_mean.empty and len(cross_perf_mean) > 1:
            cross_perf_mean["mean"] = cross_perf_mean.mean(axis=1)
            cross_perf_mean = cross_perf_mean.sort_values("mean", ascending=False).drop("mean", axis=1)
            cross_perf_std = cross_perf_std.reindex(cross_perf_mean.index)

            fig_cross_dataset, ax = plt.subplots(figsize=(9, 6))

            x = np.arange(len(cross_perf_mean))
            width = 0.35

            colors = ["#3498db", "#e74c3c"]
            for i, col in enumerate(cross_perf_mean.columns):
                yerr = cross_perf_std[col] if col in cross_perf_std.columns else None
                ax.bar(
                    x + i * width,
                    cross_perf_mean[col],
                    width,
                    label=col,
                    color=colors[i],
                    alpha=0.8,
                    yerr=yerr,
                    capsize=4,
                    error_kw={"elinewidth": 1, "ecolor": "#333333"},
                )

            ax.set_ylabel("Mean Test Pearson")
            ax.set_title(f"Cross-Dataset Model Performance Comparison | {title_suffix}")
            ax.set_xticks(x + width / 2)
            ax.set_xticklabels([to_short_name(mid) for mid in cross_perf_mean.index], rotation=45, ha="right")
            ax.legend()
            ax.grid(axis="y", alpha=0.3)
            sns.despine(fig_cross_dataset, left=True, bottom=True)
            fig_cross_dataset.tight_layout()
            register_figure(FIGURES, "cross_dataset_performance", fig_cross_dataset)
            display(fig_cross_dataset)
            plt.close(fig_cross_dataset)
        else:
            print("Insufficient models present in both datasets for cross-dataset analysis.")
    else:
        print("Cross-dataset comparison unavailable (single dataset or incomplete data).")
else:
    print("Summary data unavailable for cross-dataset analysis.")



## 15. Model Ranking Stability Across Splits


In [ ]:
# Model ranking stability across splits (per dataset)
summary_reset = analysis_state["summary_reset"].copy()
summary_reset["dataset"] = summary_reset["run_name"].map(_assign_dataset)

title_suffix = RUN_SUBSET_DESCRIPTION or f"{GENE_COUNT} genes"

if not summary_reset.empty and "test_pearson_mean" in summary_reset and "val_pearson_mean" in summary_reset:
    for dataset in ["Embryonic", "Endothelial"]:
        rank_data = summary_reset[summary_reset["dataset"] == dataset][["model_display", "test_pearson_mean", "val_pearson_mean"]].copy()
        rank_data = rank_data.dropna()
        if rank_data.empty or len(rank_data) <= 2:
            continue
        rank_data["test_rank"] = rank_data["test_pearson_mean"].rank(ascending=False)
        rank_data["val_rank"] = rank_data["val_pearson_mean"].rank(ascending=False)
        rank_data = rank_data.sort_values("test_rank")

        fig_ranking, ax = plt.subplots(figsize=(10, 6))
        x = np.arange(len(rank_data))
        width = 0.35
        ax.bar(x - width/2, rank_data["test_rank"], width, label="Test Set Rank", color="#2ecc71", alpha=0.8)
        ax.bar(x + width/2, rank_data["val_rank"], width, label="Val Set Rank", color="#f39c12", alpha=0.8)
        ax.set_ylabel("Rank (lower is better)")
        ax.set_title(f"{dataset} | Model Ranking Stability (Test vs Val) | {title_suffix}")
        ax.set_xticks(x)
        ax.set_xticklabels([to_short_name(name) for name in rank_data["model_display"]], rotation=45, ha="right")
        ax.invert_yaxis()
        ax.legend()
        sns.despine(fig_ranking, left=True, bottom=True)
        fig_ranking.tight_layout()
        register_figure(FIGURES, f"model_ranking_stability_{dataset.lower()}", fig_ranking)
        display(fig_ranking)
        plt.close(fig_ranking)



## 16. Model Complexity vs Performance


In [ ]:
# Model complexity vs performance (simple plot)
fig_complexity = None
if not summary_reset.empty and "test_pearson_mean" in summary_reset:
    run_df = analysis_state.get("run_df")
    complexity_data = summary_reset.copy()

    if "param_count" not in complexity_data.columns:
        param_count_path = config.project_root / "analysis" / "param_count.csv"
        if param_count_path.exists():
            try:
                param_count_df = pd.read_csv(param_count_path)
            except Exception:
                param_count_df = None
            if isinstance(param_count_df, pd.DataFrame) and not param_count_df.empty:
                expected = {"model_display", "model_id", "dataset", "param_count"}
                if expected.issubset(param_count_df.columns):
                    param_count_df = param_count_df.drop_duplicates(["model_id", "dataset"])
                    complexity_data = complexity_data.merge(
                        param_count_df[["model_id", "dataset", "param_count"]],
                        on=["model_id", "dataset"],
                        how="left",
                    )
        if "param_count" not in complexity_data.columns:
            complexity_data["param_count"] = 1

    if complexity_data["param_count"].isna().any() and isinstance(run_df, pd.DataFrame) and not run_df.empty:
        def _select_model_path(model_dir: Path) -> Path | None:
            if not model_dir or not model_dir.exists():
                return None
            candidates = [
                p for p in model_dir.glob("model.*")
                if p.is_file() and "scaler" not in p.name.lower()
            ]
            if not candidates:
                patterns = ("*.pkl", "*.joblib", "*.sav", "*.pt", "*.pth", "*.cbm", "*.json", "*.bst")
                for pattern in patterns:
                    candidates.extend(
                        p for p in model_dir.glob(pattern)
                        if p.is_file() and "scaler" not in p.name.lower()
                    )
            if not candidates:
                return None
            candidates.sort(key=lambda p: (p.name != "model.pkl", p.name))
            return candidates[0]

        def _approx_param_count(model_path: Path | None) -> int:
            if not model_path or not model_path.exists():
                return 1
            try:
                return max(1, int(model_path.stat().st_size))
            except Exception:
                return 1

        approx_counts = []
        for _, row in complexity_data.iterrows():
            count = row.get("param_count")
            if pd.notna(count):
                approx_counts.append(int(count))
                continue
            match = run_df[(run_df["run_name"] == row["run_name"]) & (run_df["model_id"] == row["model_id"])]
            if match.empty:
                approx_counts.append(1)
                continue
            model_dir = match.iloc[0]["model_path"]
            model_dir = model_dir if isinstance(model_dir, Path) else Path(model_dir)
            model_path = _select_model_path(model_dir)
            approx_counts.append(_approx_param_count(model_path))
        complexity_data["param_count"] = approx_counts

    param_table = (
        complexity_data[["model_display", "model_id", "dataset", "param_count", "test_pearson_mean"]]
        .drop_duplicates(["model_id", "dataset"])
        .sort_values(["dataset", "param_count"], ascending=[True, False])
        .reset_index(drop=True)
    )
    display(param_table)

    fig_complexity, ax = plt.subplots(figsize=(22, 12))
    colors = {"Embryonic": "#4C78A8", "Endothelial": "#F58518"}

    label_nudges = {
        # (model_id, dataset): (dx, dy) in points
        ("elastic_net", "Embryonic"): (9, 9),
        ("lasso", "Embryonic"): (9, -9),
        ("extra_trees", "Endothelial"): (-55, 0),
        ("hist_gradient_boosting", "Embryonic"): (6, 6),
        ("cnn", "Endothelial"): (9, -9),
        ("random_forest", "Embryonic"): (12, 0),
        ("xgboost", "Embryonic"): (9, 7),
        ("ridge", "Embryonic"): (9, 9),
        ("ols", "Embryonic"): (9, -9),
        ("ols", "Endothelial"): (9, 3),
        ("ridge", "Endothelial"): (9, -9),
    }





















    for dataset in ["Embryonic", "Endothelial"]:
        subset = complexity_data[complexity_data["dataset"] == dataset].copy()
        if subset.empty:
            continue
        ax.scatter(
            subset["param_count"],
            subset["test_pearson_mean"],
            s=110,
            alpha=0.8,
            color=colors.get(dataset, "#666666"),
            edgecolors="black",
            linewidth=0.6,
            label=dataset,
        )
        default_dx, default_dy = 6, 6
        for row in subset.itertuples(index=False):
            label_key = (getattr(row, "model_id"), dataset)
            dx, dy = label_nudges.get(label_key, (default_dx, default_dy))
            ax.annotate(
                to_short_name(getattr(row, "model_display")),
                (getattr(row, "param_count"), getattr(row, "test_pearson_mean")),
                xytext=(dx, dy),
                textcoords="offset points",
                fontsize=8,
                alpha=0.9,
                bbox=dict(boxstyle="round,pad=0.15", facecolor="white", edgecolor="none", alpha=0.55),
            )

    ax.set_xlabel("Parameter count (approx; log scale)")
    ax.set_ylabel("Mean Test Pearson")
    ax.set_title(f"Model Complexity vs Performance | {title_suffix}")
    ax.set_xscale("log")
    ax.grid(True, alpha=0.3, axis="both")
    ax.legend(loc="upper right", frameon=True)
    sns.despine(fig_complexity, left=True, bottom=True)
    fig_complexity.subplots_adjust(left=0.08, right=0.96, top=0.90, bottom=0.15)
    register_figure(FIGURES, "complexity_vs_performance", fig_complexity)
    display(fig_complexity)
    plt.close(fig_complexity)



## 17. Export Figures

Save all generated matplotlib figures to the manuscript figure directory.


In [ ]:
import shutil

project_root = Path(analysis_metadata.get("project_root", config.project_root))
fig_dir_raw = analysis_metadata.get("fig_dir", config.fig_dir)
fig_dir = Path(fig_dir_raw)
if not fig_dir.is_absolute():
    fig_dir = project_root / fig_dir
fig_dir = fig_dir.resolve()
fig_dir.mkdir(parents=True, exist_ok=True)

exported = []
if not FIGURES:
    print("No figures registered for export.")
else:
    for key, fig in FIGURES.items():
        if fig is None:
            continue
        out_path = fig_dir / f"{key}.png"
        fig.savefig(out_path, bbox_inches="tight")
        exported.append(out_path)

if EXTRA_EXPORT_PATHS:
    for src_path, out_name in EXTRA_EXPORT_PATHS:
        src_path = Path(src_path)
        if not src_path.exists():
            continue
        out_path = fig_dir / out_name
        shutil.copy2(src_path, out_path)
        exported.append(out_path)

if not exported:
    print("No figures were exported.")
else:
    print(f"Exported {len(exported)} figures to {fig_dir}")
    for path in exported:
        print(f"- {path}")
